# Final Project Code
## Outline
1. Getting products from Trader Joe's
2. Getting products from SaveMart
3. Getting products from Safeway

In [2]:
import requests
import lxml.html as lx
import pandas as pd
import time
import re
import numpy as np
import matplotlib.pyplot as plt

## Trader Joe's
Getting products from Trader Joe's

In [ ]:
# function creates a dataframe w/ all the product information
# based on a search
# arguments: the ingredient name
def tj_info(ingredient:str):
    url = "https://www.traderjoes.com/api/graphql"
    i = 1 # counting parameter
    # initialize list to store all results
    results = []
    # run through the pages of the search results and save results to the list
    # realized that we needed this after butter didn't have relevant results until the later on
    while True:
        query = {"operationName":"SearchProducts",
                 "variables":{"storeCode":"182","availability":"1","published":"1","search":ingredient,"currentPage":i,"pageSize":15},
                 "query":"query SearchProducts($search: String, $pageSize: Int, $currentPage: Int, $storeCode: String = \"182\", $availability: String = \"1\", $published: String = \"1\") {\n  products(\n    search: $search\n    filter: {store_code: {eq: $storeCode}, published: {eq: $published}, availability: {match: $availability}}\n    pageSize: $pageSize\n    currentPage: $currentPage\n  ) {\n    items {\n      category_hierarchy {\n        id\n        url_key\n        description\n        name\n        position\n        level\n        created_at\n        updated_at\n        product_count\n        __typename\n      }\n      item_story_marketing\n      product_label\n      fun_tags\n      primary_image\n      primary_image_meta {\n        url\n        metadata\n        __typename\n      }\n      other_images\n      other_images_meta {\n        url\n        metadata\n        __typename\n      }\n      context_image\n      context_image_meta {\n        url\n        metadata\n        __typename\n      }\n      published\n      sku\n      url_key\n      name\n      item_description\n      item_title\n      item_characteristics\n      item_story_qil\n      use_and_demo\n      sales_size\n      sales_uom_code\n      sales_uom_description\n      country_of_origin\n      availability\n      new_product\n      promotion\n      price_range {\n        minimum_price {\n          final_price {\n            currency\n            value\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      retail_price\n      nutrition {\n        display_sequence\n        panel_id\n        panel_title\n        serving_size\n        calories_per_serving\n        servings_per_container\n        details {\n          display_seq\n          nutritional_item\n          amount\n          percent_dv\n          __typename\n        }\n        __typename\n      }\n      ingredients {\n        display_sequence\n        ingredient\n        __typename\n      }\n      allergens {\n        display_sequence\n        ingredient\n        __typename\n      }\n      created_at\n      first_published_date\n      last_published_date\n      updated_at\n      related_products {\n        sku\n        item_title\n        primary_image\n        primary_image_meta {\n          url\n          metadata\n          __typename\n        }\n        price_range {\n          minimum_price {\n            final_price {\n              currency\n              value\n              __typename\n            }\n            __typename\n          }\n          __typename\n        }\n        retail_price\n        sales_size\n        sales_uom_description\n        category_hierarchy {\n          id\n          name\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    total_count\n    page_info {\n      current_page\n      page_size\n      total_pages\n      __typename\n    }\n    __typename\n  }\n}\n"}
        time.sleep(.5) # try not to get banned
        # retrieve website information
        response = requests.post(url,json=query)
        # check for error
        response.raise_for_status()
        res = response.json() # convert to json
        # if no products are returned
        # escape the loops
        if not res["data"]["products"]["items"]:
            break
        results.append(res)
        i += 1 # increment counting parameter
        print(i) # diagnostic statement
    # create list to store product information
    prod = [[
        subitem["item_title"],
        subitem["retail_price"],
        subitem["sales_size"],
        subitem["sales_uom_description"]
        ] for item in results for subitem in item["data"]["products"]["items"]]
    # organize results into dataframe
    df = pd.DataFrame(data=prod,columns=[
        "Product Name","Retail Price","Sales Size","Unit of Measurement"
    ])
    return df

### All-Purpose Flour

In [ ]:
flo_df = tj_info("all-purpose flour") # get flour information
flo_df

In [ ]:
# filter for results w/ the key word all-purpose flour
flo_fil = flo_df[ # check enough quantity, which is 3 cups
    ((flo_df['Sales Size'] >= 3*0.275578) & (flo_df['Unit of Measurement'].str.contains('Lb'))) &
    (flo_df['Product Name'].str.contains('All-Purpose Flour', case=False, na=False))
]
# retrieve the minimum price
flo_fil.sort_values(ascending=True,by="Retail Price").head(1)

,Product Name,Retail Price,Sales Size,Unit of Measurement
0,Unbleached Enriched All-Purpose Flour,2.99,5.0,Lb


### Butter

In [ ]:
but_df = tj_info("butter") # get butter information
but_df

In [ ]:
# filter for results w/ the key word unsalted
but_fil = but_df[ # check enough quantity, which is 2 sticks = 8 oz
    ((but_df['Sales Size'] >= 8) & (but_df['Unit of Measurement'].str.contains('Oz'))) &
    (but_df['Product Name'].str.startswith('Butter')) &
    (but_df['Product Name'].str.contains('Unsalted', case=False, na=False))
]
# retrieve the minimum price
but_fil.sort_values(ascending=True,by="Retail Price").head(1)

,Product Name,Retail Price,Sales Size,Unit of Measurement
18,"Butter Quarters, Unsalted",3.99,16.0,Oz


### Sugar

In [ ]:
sug_df = tj_info("white sugar") # get sugar info
sug_df

In [ ]:
# filter for results w/ the key word sugar, cane, granulated, etc.
# don't want powdered sugar
sug_fil = sug_df[ # check enough quantity, which is 1 cup
    ((sug_df['Sales Size'] >= .44) & (sug_df['Unit of Measurement'].str.contains('Lb'))) &
    (sug_df['Product Name'].str.contains('Cane', case=False, na=False)) &
    ~(sug_df['Product Name'].str.contains('Powdered', case=False, na=False))
]
# retrieve the minimum price
sug_fil.sort_values(ascending=True,by="Retail Price").head(1)

,Product Name,Retail Price,Sales Size,Unit of Measurement
0,Organic Cane Sugar,3.99,2.0,Lb


### Vanilla Extract

In [ ]:
van = tj_info("vanilla extract") # get vanilla extract info
van

In [ ]:
# filter for results w/ the key word extract
# only retrieve unsalted butter
van_fil = van[ # check enough quantity, which is .2 fl oz
    ((van['Sales Size'] >= .2) & (van['Unit of Measurement'].str.contains('Oz'))) &
    (van['Product Name'].str.contains('Extract', case=False, na=False))
]
# retrieve the minimum price
van_fil.sort_values(ascending=True,by="Retail Price").head(1)

,Product Name,Retail Price,Sales Size,Unit of Measurement
0,Double Fold Bourbon Vanilla Extract,7.99,3.55,Oz


### Brown Sugar

In [ ]:
brown = tj_info("brown sugar") # get brown sugar info
brown

In [ ]:
# filter for results w/ the key word brown sugar
brown_fil = brown[ # check enough quantity, which is 7.05 oz or .44 lbs
    (((brown['Sales Size'] >= 7.05) & (brown['Unit of Measurement']=='Oz')) | # we don't want liquids
     ((brown['Sales Size'] >= .44) & (brown['Unit of Measurement'].str.contains('Lb')))) &
    (brown['Product Name'].str.contains('Brown Sugar', case=False, na=False))
]
# retrieve the minimum price
brown_fil.sort_values(ascending=True,by="Retail Price")

,Product Name,Retail Price,Sales Size,Unit of Measurement


### Eggs

In [ ]:
eggs = tj_info("eggs") # get eggs info
eggs

In [ ]:
# filter for results w/ the key word large
eggs_fil = eggs[ # check enough quantity, which is 2 eggs
    ((eggs['Sales Size'] >= (2/12)) & (eggs['Unit of Measurement']=='Doz')) & # only care about actual egg products
    (eggs['Product Name'].str.contains('Large', case=False, na=False))
]
# retrieve the minimum price
eggs_fil.sort_values(ascending=True,by="Retail Price")

,Product Name,Retail Price,Sales Size,Unit of Measurement
1,Pasture Raised Large Brown Eggs,4.99,1.0,Doz
0,Organic Pasture Raised Large Brown Eggs,6.49,1.0,Doz


### Baking Soda

In [ ]:
soda = tj_info("baking soda") # get baking soda info
soda

In [ ]:
# filter for results w/ the key word baking soda
soda_fil = soda[ # check enough quantity
    (soda['Product Name'].str.contains('Baking Soda', case=False, na=False))
]
# retrieve the minimum price
# no pure baking soda products
soda_fil.sort_values(ascending=True,by="Retail Price")

,Product Name,Retail Price,Sales Size,Unit of Measurement


### Chocolate Chips

In [ ]:
chips = tj_info("Semisweet chocolate chips") # get chocolate chip info
chips

In [ ]:
# filter for semisweet chocolate chips
chips_fil = chips[ # check enough quantity, which is 12 oz = 2 cups
    (chips['Sales Size'] >= 12) & (chips['Unit of Measurement']=='Oz') &
    (chips['Product Name'].str.contains('Semi', case=False, na=False))
]
# retrieve the minimum price
chips_fil.sort_values(ascending=True,by="Retail Price")

,Product Name,Retail Price,Sales Size,Unit of Measurement
0,Semi-Sweet Chocolate Chips,3.99,12.0,Oz


### Chopped Walnuts

In [ ]:
nuts = tj_info("chopped walnuts") # get walnut info
nuts

In [ ]:
# filter for chopped walnut products
nuts_fil = nuts[ # check enough quantity
    (nuts['Sales Size'] >= 4.4) & (nuts['Unit of Measurement']=='Oz') &
    (nuts['Product Name'].str.contains('Chopped Walnuts', case=False, na=False))
]
# retrieve the minimum price
# no products w/ chopped walnuts
nuts_fil.sort_values(ascending=True,by="Retail Price")

,Product Name,Retail Price,Sales Size,Unit of Measurement


## SaveMart
Getting products from SaveMart

In [ ]:
# function to retrieve the quantity and unit from each product name
def extract_quantity_and_unit(product_name):
    # This regex handles potential spacing issues and units at the end of the string
    regex = r"(\d+(\.\d+)?)\s*(fl\.?\s?oz\.?|oz\.?|ea|lbs?\.*|g|ml|kg|sticks|box|pound|gr|cups|tbsp|bottle|jar|ct|pk|count)(?=\s|$)"
    
    # Search for the pattern in the product name
    match = re.search(regex, product_name.lower())
    # if the pattern exists, get the quantity and unit
    if match:
        quantity = match.group(1) # the actual number
        unit = match.group(3) # unit of measurement
        return quantity, unit
    else:
        return None, None

# get product information from each search
# and organize into dataframe
# query: copied from the developer tools tab
def get_info(query:dict):
    url = "https://savemart.com/graphql"
    # get information from savemart
    response = requests.post(url,json=query)
    response.raise_for_status() # check for error
    response = response.json() # convert to json
    # initialize list to store product details
    prod = []
    # run through each product and add to prod list
    for item in response["data"]["products"]["products"]:
        # Check if "webPrice" exists and is not None
        if item.get("webPrice") and item["webPrice"].get("success"):
            # Access the basePrice only if it's safe to do so
            # base price is the price without any sales
            # from the base price key
            price = item["webPrice"]["success"].get("basePrice")
            # retrieve the price
            try: # most prices are in $ format
                actual_price = re.findall(r'\$(.*?)</span>', price)[0]
            except: # handle the price w/ 99¢
                actual_price = re.findall(r'>(.*?)</span>', price)[0]
                # strip the 99¢
                actual_price = "0."+actual_price.strip("¢")
            # retrieve quantity and unit
            try: 
                qty,unit = extract_quantity_and_unit(item["title"])
            except: # return none if fails
                qty = None
                unit = None
            # update list
            prod.append([item["brand"],actual_price,item["title"],qty,unit])
        else: # if no webprice or success, nothing is returned
            print("Price details not available")
    # create dataframe w/ results
    svmrt = pd.DataFrame(data=prod,columns=[
        "Brand","Price","Name","Quantity","Unit"
    ])
    # convert columns to numeric
    svmrt["Price"] = pd.to_numeric(svmrt['Price'], errors='coerce')
    svmrt["Quantity"] = pd.to_numeric(svmrt['Quantity'], errors='coerce')
    return svmrt # return the dataframe

### Butter

In [ ]:
query = {"query":"\n    query MerchandisedCategory($adCapabilityPackId: AdCapabilityPackId!, $networkId: String!, $siteId: String!, $limit: Int!, $offset: Int!, $taxonomyNodeId: String!, $adsScreenName: String!, $cookie: String, $sortBy: ProductSortDimension, $productQueryMatch: ProductMatchInput!, $storeId: String) {\n  heroTopAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroTop\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroTop\"\n  ) {\n    ...ad_ref_list\n  }\n  heroBottomAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroBottom\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroBottom\"\n  ) {\n    ...ad_ref_list\n  }\n  browseSubcategories(id: $taxonomyNodeId) @skip(if: true)\n  navOptionsCategories(id: $taxonomyNodeId)\n  taxonomy(id: \"Mobile/P+C\") @skip(if: true) {\n    __typename\n    ...taxonomy\n  }\n  merchandisedDisplayableObjects(\n    id: $taxonomyNodeId\n    taxonomyId: \"Mobile/P+C\"\n    positions: [$adsScreenName]\n  ) @skip(if: true) {\n    __typename\n    ... on DisplayableAd {\n      ...displayable_ad\n    }\n    ... on CategoryPreview {\n      __typename\n      ads {\n        ...ad_with_placement\n      }\n      categoryId\n      ... on ProductCategoryPreview {\n        items {\n          ...product_summary\n        }\n        prefixItems {\n          ...product_summary_with_ad_telemetry\n        }\n      }\n    }\n  }\n  products(\n    limit: $limit\n    cookie: $cookie\n    offset: $offset\n    taxonomyId: \"Mobile/P+C\"\n    sortBy: $sortBy\n    match: $productQueryMatch\n    storeId: $storeId\n  ) {\n    ...product_result\n  }\n}\n    \n    fragment ad_ref_list on AdRefList {\n  __typename\n  adRef\n  adList {\n    ...ad_with_placement\n  }\n}\n    \n\n    fragment ad_with_placement on AdWithPlacement {\n  __typename\n  position\n  ad {\n    __typename\n    ...ad_with_telemetry\n  }\n  placementId\n}\n    \n\n    fragment ad_with_telemetry on AdDTO {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  aid\n  aiid\n  height\n  pid\n  width\n  version\n  ... on CollectionAdDTO {\n    placements {\n      ...collection_ad_placement\n    }\n    ... on CarouselCollectionDTO {\n      randomizationStyle\n    }\n    title\n    subtitle\n  }\n  ... on ImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on ImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n  ... on LayoutAdDTO {\n    rows {\n      __typename\n      height\n      columns {\n        __typename\n        width\n        adRef\n      }\n    }\n  }\n  ... on RoundedImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on RoundedImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n}\n    \n\n    fragment ad_telemetry on AdTelemetry {\n  __typename\n  aid\n  aiid\n  dimensions {\n    __typename\n    key\n    value\n  }\n}\n    \n\n    fragment collection_ad_placement on PlacementDTO {\n  __typename\n  height\n  id\n  width\n  ... on LeafPlacementDTO {\n    adRef\n    ... on CarouselPlacementDTO {\n      displayDurationMs\n    }\n  }\n  ... on LayoutRowPlacementDTO {\n    placements {\n      ... on LeafPlacementDTO {\n        adRef\n        ... on CarouselPlacementDTO {\n          displayDurationMs\n        }\n      }\n    }\n  }\n}\n    \n\n    fragment ad_image_asset on ImageAsset {\n  __typename\n  id\n  densities {\n    __typename\n    swiftlyDensity\n    swiftlyDeviceClass\n    url\n  }\n  altText\n  revision\n  action {\n    __typename\n    action\n    name\n    target\n  }\n}\n    \n\n    fragment carousel_page on CarouselPage {\n  __typename\n  durationMs\n  imageAsset {\n    __typename\n    ...ad_image_asset\n  }\n}\n    \n\n    fragment taxonomy on Taxonomy {\n  allowedTypes\n  displayName\n  id\n  graph {\n    children\n    displayName\n    id\n    parents\n    renderingTemplate\n    type\n    images {\n      ...image_ref\n    }\n  }\n}\n    \n\n    fragment image_ref on ImageRef {\n  __typename\n  altText\n  caption\n  image(desiredDensity: ThreeX) {\n    ...image_file\n  }\n  images(desiredDensities: [ThreeX]) {\n    ...image_file\n  }\n  type {\n    ...image_type\n  }\n}\n    \n\n    fragment image_file on ImageFile {\n  __typename\n  density\n  device\n  uri\n}\n    \n\n    fragment image_type on ImageType {\n  __typename\n  description\n  height\n  type\n  width\n}\n    \n\n    fragment displayable_ad on DisplayableAd {\n  ad {\n    ...ad_ref_list\n  }\n}\n    \n\n    fragment product_summary on ProductSummary {\n  __typename\n  brand\n  categories\n  eligibilities {\n    __typename\n    eligibilityId\n  }\n  legacyPriceTag: fdPriceTag {\n    ...legacy_price_tag\n  }\n  id\n  images {\n    ...image_ref\n  }\n  ordinal\n  webPrice {\n    failure {\n      type\n      message\n      displayMessage\n    }\n    success {\n      basePrice\n      promoPrice\n      promoText\n      isSale\n    }\n  }\n  price {\n    ...price_result\n  }\n  shortDescription\n  tags {\n    ...product_tag\n  }\n  title\n  type\n}\n    \n\n    fragment legacy_price_tag on FamilyDollarPriceTag {\n  __typename\n  discountType\n  finalPrice {\n    ...legacy_price_detail\n  }\n  finalSavings {\n    ...legacy_savings_detail\n  }\n  priceUri\n  regularPrice {\n    ...legacy_price_detail\n  }\n  scenario\n  tags {\n    ...product_tag\n  }\n  termsAndConditions\n}\n    \n\n    fragment legacy_price_detail on PriceDetail {\n  __typename\n  buyQuantity\n  displayPrice\n  effectiveDate\n  expirationDate\n  getQuantity\n  mixAndMatchId\n  model\n  price\n  quantityLimit\n  quantityMinimum\n  soldBy\n  type\n}\n    \n\n    fragment legacy_savings_detail on SavingsDetail {\n  __typename\n  mustBuyAtLeast\n  savingsAmount\n  savingsDisplayAmount\n  savingsDisplayPercent\n  savingsPercent\n  savingsQuantity\n}\n    \n\n    fragment product_tag on Tag {\n  __typename\n  code\n  description\n  id\n  images {\n    ...image_ref\n  }\n  title\n  type\n  value\n}\n    \n\n    fragment price_result on PriceResult {\n  __typename\n  failure {\n    __typename\n    message\n    type\n  }\n  success {\n    ...price\n  }\n}\n    \n\n    fragment price on Price {\n  __typename\n  base {\n    ...price_model_properties\n  }\n  cost {\n    ...cost\n  }\n  info {\n    ...price_info\n  }\n  mixAndMatchId\n  promotion {\n    ...price_model_properties\n  }\n  savings {\n    ...savings\n  }\n  scenario\n  soldBy\n  unit\n}\n    \n\n    fragment price_model_properties on PriceModelProperties {\n  __typename\n  activeThrough {\n    ...active_through\n  }\n  amountOff {\n    ...money\n  }\n  buyQuantity\n  getQuantity\n  model\n  percentOff\n  price {\n    ...money\n  }\n  type\n}\n    \n\n    fragment active_through on ActiveThrough {\n  __typename\n  end {\n    ...active_date\n  }\n  start {\n    ...active_date\n  }\n}\n    \n\n    fragment active_date on ActiveDate {\n  __typename\n  date\n  time\n}\n    \n\n    fragment money on Money {\n  __typename\n  currency\n  denomination\n  value\n}\n    \n\n    fragment cost on Cost {\n  __typename\n  purchaseQuantity\n  totalCost {\n    ...money\n  }\n  type\n}\n    \n\n    fragment price_info on PriceInfo {\n  __typename\n  code\n  id\n  type\n  value\n}\n    \n\n    fragment savings on Savings {\n  __typename\n  percentSaved\n  savingsQuantity\n  totalSavingsAmount {\n    ...money\n  }\n  type\n}\n    \n\n    fragment product_summary_with_ad_telemetry on ProductSummaryWithAdTelemetry {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  product {\n    ...product_summary\n  }\n}\n    \n\n    fragment product_result on ProductResult {\n  __typename\n  cookie\n  count\n  fieldFacets {\n    ...field_facet\n  }\n  overlapCount\n  prefixCount\n  prefixProducts {\n    ...product_summary_with_ad_telemetry\n  }\n  products {\n    ...product_summary\n  }\n  spelling {\n    __typename\n    corrected\n    original\n  }\n}\n    \n\n    fragment field_facet on FieldFacet {\n  __typename\n  displayName\n  entries {\n    ...field_facet_entry\n  }\n  field\n}\n    \n\n    fragment field_facet_entry on FieldFacetEntry {\n  __typename\n  count\n  entry\n}\n    ","variables":{"adCapabilityPackId":"AD_CARACAL_1","adsScreenName":"ProductSearch","limit":50,"offset":0,"taxonomyNodeId":"Mobile/P+C/Product/RootCategoryId","networkId":"TSMC0.0000000002","siteId":"sm-shopperWeb","cookie":"","sortBy":"Featured","storeId":"604","productQueryMatch":{"search":{"keyword":"butter"}}}}
butter = get_info(query) # get butter products from SaveMart

In [6]:
# only retrieve unsalted butter
butter_svmrt = butter[ # check enough quantity
    (((butter['Quantity'] >= 16) & (butter['Unit'].str.contains('oz'))) | # 16 oz case
    ((butter['Quantity'] >= 4) & ((butter['Unit'].str.contains('ea')) | (butter['Unit'].str.contains('sticks'))))) & # 4 sticks/ea case
    ((butter['Name'].str.contains('Unsalted', case=False, na=False)))
]
# retrieve the minimum price
butter_svmrt.sort_values(ascending=True,by="Price").head(1)

,Brand,Price,Name,Quantity,Unit
24,SUNNYSIDE FARMS,5.99,Unsalted Sweet Cream Butter 4 ea,4.0,ea


### Sugar

In [ ]:
query = {"query":"\n    query MerchandisedCategory($adCapabilityPackId: AdCapabilityPackId!, $networkId: String!, $siteId: String!, $limit: Int!, $offset: Int!, $taxonomyNodeId: String!, $adsScreenName: String!, $cookie: String, $sortBy: ProductSortDimension, $productQueryMatch: ProductMatchInput!, $storeId: String) {\n  heroTopAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroTop\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroTop\"\n  ) {\n    ...ad_ref_list\n  }\n  heroBottomAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroBottom\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroBottom\"\n  ) {\n    ...ad_ref_list\n  }\n  browseSubcategories(id: $taxonomyNodeId) @skip(if: true)\n  navOptionsCategories(id: $taxonomyNodeId)\n  taxonomy(id: \"Mobile/P+C\") @skip(if: true) {\n    __typename\n    ...taxonomy\n  }\n  merchandisedDisplayableObjects(\n    id: $taxonomyNodeId\n    taxonomyId: \"Mobile/P+C\"\n    positions: [$adsScreenName]\n  ) @skip(if: true) {\n    __typename\n    ... on DisplayableAd {\n      ...displayable_ad\n    }\n    ... on CategoryPreview {\n      __typename\n      ads {\n        ...ad_with_placement\n      }\n      categoryId\n      ... on ProductCategoryPreview {\n        items {\n          ...product_summary\n        }\n        prefixItems {\n          ...product_summary_with_ad_telemetry\n        }\n      }\n    }\n  }\n  products(\n    limit: $limit\n    cookie: $cookie\n    offset: $offset\n    taxonomyId: \"Mobile/P+C\"\n    sortBy: $sortBy\n    match: $productQueryMatch\n    storeId: $storeId\n  ) {\n    ...product_result\n  }\n}\n    \n    fragment ad_ref_list on AdRefList {\n  __typename\n  adRef\n  adList {\n    ...ad_with_placement\n  }\n}\n    \n\n    fragment ad_with_placement on AdWithPlacement {\n  __typename\n  position\n  ad {\n    __typename\n    ...ad_with_telemetry\n  }\n  placementId\n}\n    \n\n    fragment ad_with_telemetry on AdDTO {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  aid\n  aiid\n  height\n  pid\n  width\n  version\n  ... on CollectionAdDTO {\n    placements {\n      ...collection_ad_placement\n    }\n    ... on CarouselCollectionDTO {\n      randomizationStyle\n    }\n    title\n    subtitle\n  }\n  ... on ImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on ImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n  ... on LayoutAdDTO {\n    rows {\n      __typename\n      height\n      columns {\n        __typename\n        width\n        adRef\n      }\n    }\n  }\n  ... on RoundedImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on RoundedImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n}\n    \n\n    fragment ad_telemetry on AdTelemetry {\n  __typename\n  aid\n  aiid\n  dimensions {\n    __typename\n    key\n    value\n  }\n}\n    \n\n    fragment collection_ad_placement on PlacementDTO {\n  __typename\n  height\n  id\n  width\n  ... on LeafPlacementDTO {\n    adRef\n    ... on CarouselPlacementDTO {\n      displayDurationMs\n    }\n  }\n  ... on LayoutRowPlacementDTO {\n    placements {\n      ... on LeafPlacementDTO {\n        adRef\n        ... on CarouselPlacementDTO {\n          displayDurationMs\n        }\n      }\n    }\n  }\n}\n    \n\n    fragment ad_image_asset on ImageAsset {\n  __typename\n  id\n  densities {\n    __typename\n    swiftlyDensity\n    swiftlyDeviceClass\n    url\n  }\n  altText\n  revision\n  action {\n    __typename\n    action\n    name\n    target\n  }\n}\n    \n\n    fragment carousel_page on CarouselPage {\n  __typename\n  durationMs\n  imageAsset {\n    __typename\n    ...ad_image_asset\n  }\n}\n    \n\n    fragment taxonomy on Taxonomy {\n  allowedTypes\n  displayName\n  id\n  graph {\n    children\n    displayName\n    id\n    parents\n    renderingTemplate\n    type\n    images {\n      ...image_ref\n    }\n  }\n}\n    \n\n    fragment image_ref on ImageRef {\n  __typename\n  altText\n  caption\n  image(desiredDensity: ThreeX) {\n    ...image_file\n  }\n  images(desiredDensities: [ThreeX]) {\n    ...image_file\n  }\n  type {\n    ...image_type\n  }\n}\n    \n\n    fragment image_file on ImageFile {\n  __typename\n  density\n  device\n  uri\n}\n    \n\n    fragment image_type on ImageType {\n  __typename\n  description\n  height\n  type\n  width\n}\n    \n\n    fragment displayable_ad on DisplayableAd {\n  ad {\n    ...ad_ref_list\n  }\n}\n    \n\n    fragment product_summary on ProductSummary {\n  __typename\n  brand\n  categories\n  eligibilities {\n    __typename\n    eligibilityId\n  }\n  legacyPriceTag: fdPriceTag {\n    ...legacy_price_tag\n  }\n  id\n  images {\n    ...image_ref\n  }\n  ordinal\n  webPrice {\n    failure {\n      type\n      message\n      displayMessage\n    }\n    success {\n      basePrice\n      promoPrice\n      promoText\n      isSale\n    }\n  }\n  price {\n    ...price_result\n  }\n  shortDescription\n  tags {\n    ...product_tag\n  }\n  title\n  type\n}\n    \n\n    fragment legacy_price_tag on FamilyDollarPriceTag {\n  __typename\n  discountType\n  finalPrice {\n    ...legacy_price_detail\n  }\n  finalSavings {\n    ...legacy_savings_detail\n  }\n  priceUri\n  regularPrice {\n    ...legacy_price_detail\n  }\n  scenario\n  tags {\n    ...product_tag\n  }\n  termsAndConditions\n}\n    \n\n    fragment legacy_price_detail on PriceDetail {\n  __typename\n  buyQuantity\n  displayPrice\n  effectiveDate\n  expirationDate\n  getQuantity\n  mixAndMatchId\n  model\n  price\n  quantityLimit\n  quantityMinimum\n  soldBy\n  type\n}\n    \n\n    fragment legacy_savings_detail on SavingsDetail {\n  __typename\n  mustBuyAtLeast\n  savingsAmount\n  savingsDisplayAmount\n  savingsDisplayPercent\n  savingsPercent\n  savingsQuantity\n}\n    \n\n    fragment product_tag on Tag {\n  __typename\n  code\n  description\n  id\n  images {\n    ...image_ref\n  }\n  title\n  type\n  value\n}\n    \n\n    fragment price_result on PriceResult {\n  __typename\n  failure {\n    __typename\n    message\n    type\n  }\n  success {\n    ...price\n  }\n}\n    \n\n    fragment price on Price {\n  __typename\n  base {\n    ...price_model_properties\n  }\n  cost {\n    ...cost\n  }\n  info {\n    ...price_info\n  }\n  mixAndMatchId\n  promotion {\n    ...price_model_properties\n  }\n  savings {\n    ...savings\n  }\n  scenario\n  soldBy\n  unit\n}\n    \n\n    fragment price_model_properties on PriceModelProperties {\n  __typename\n  activeThrough {\n    ...active_through\n  }\n  amountOff {\n    ...money\n  }\n  buyQuantity\n  getQuantity\n  model\n  percentOff\n  price {\n    ...money\n  }\n  type\n}\n    \n\n    fragment active_through on ActiveThrough {\n  __typename\n  end {\n    ...active_date\n  }\n  start {\n    ...active_date\n  }\n}\n    \n\n    fragment active_date on ActiveDate {\n  __typename\n  date\n  time\n}\n    \n\n    fragment money on Money {\n  __typename\n  currency\n  denomination\n  value\n}\n    \n\n    fragment cost on Cost {\n  __typename\n  purchaseQuantity\n  totalCost {\n    ...money\n  }\n  type\n}\n    \n\n    fragment price_info on PriceInfo {\n  __typename\n  code\n  id\n  type\n  value\n}\n    \n\n    fragment savings on Savings {\n  __typename\n  percentSaved\n  savingsQuantity\n  totalSavingsAmount {\n    ...money\n  }\n  type\n}\n    \n\n    fragment product_summary_with_ad_telemetry on ProductSummaryWithAdTelemetry {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  product {\n    ...product_summary\n  }\n}\n    \n\n    fragment product_result on ProductResult {\n  __typename\n  cookie\n  count\n  fieldFacets {\n    ...field_facet\n  }\n  overlapCount\n  prefixCount\n  prefixProducts {\n    ...product_summary_with_ad_telemetry\n  }\n  products {\n    ...product_summary\n  }\n  spelling {\n    __typename\n    corrected\n    original\n  }\n}\n    \n\n    fragment field_facet on FieldFacet {\n  __typename\n  displayName\n  entries {\n    ...field_facet_entry\n  }\n  field\n}\n    \n\n    fragment field_facet_entry on FieldFacetEntry {\n  __typename\n  count\n  entry\n}\n    ","variables":{"adCapabilityPackId":"AD_CARACAL_1","adsScreenName":"ProductSearch","limit":50,"offset":0,"taxonomyNodeId":"Mobile/P+C/Product/RootCategoryId","networkId":"TSMC0.0000000002","siteId":"sm-shopperWeb","cookie":"","sortBy":"Featured","storeId":"604","productQueryMatch":{"search":{"keyword":"white sugar"}}}}
sugar = get_info(query) # get sugar products from SaveMart

In [55]:
sugar

,Brand,Price,Name,Quantity,Unit
0,SUNNY SELECT,2.19,Enriched Long grain White Rice 32 oz,32.00,oz
1,SUGAR IN THE RAW,5.99,White Organic Cane Sugar 24.0 oz,24.00,oz
2,SUNNY SELECT,4.29,Enriched Long Grain White Rice 5 oz,5.00,oz
3,SKINNY POP POPCORN,5.49,Skinny Pop White Cheddar Popcorn 4.4 oz,4.40,oz
4,C & H SUGAR,3.99,C&H® Pure Cane Granulated White Sugar 32 oz. Box,32.00,oz
5,HUNGRY-MAN FROZEN DINNERS,5.99,Hungry Man Roasted Carved White Meat Turkey 16 oz,16.00,oz
6,ROCK STAR ENERGY DRINKS,3.29,Rockstar Focus Sugar Free Energy Drink White P...,12.00,fl oz
7,ZOA ENERGY,2.69,Zoa Zero Sugar White Peach Energy Drink 12 fl oz,12.00,fl oz
8,BIRD'S EYE,3.99,"Birds Eye Steamfresh Gold & White Corn, Carrot...",10.80,oz
9,ZULKA HISPANIC SUGAR,5.99,Zulka Sugar 4 lb,4.00,lb


In [ ]:
# filter for relevant sugars
sugar_svmrt = sugar[
    ((sugar['Name'].str.contains('Sugar', case=False, na=False)) & ( # should actually have the word sugar
        ~(sugar['Name'].str.contains('Drink', case=False, na=False)) & ~( # no drinks
            sugar['Name'].str.contains('Gum', case=False, na=False) # no gum
        ) & ~(
            sugar['Name'].str.contains('Turbinado', case=False, na=False) # wrong type of sugar
        ) & ~(
            sugar['Name'].str.contains('Brown', case=False, na=False) # wrong type of sugar
        ) & ~(
            sugar['Name'].str.contains('Coconut', case=False, na=False) # wrong type of sugar
        ) & ~(
            sugar['Name'].str.contains('Chicken', case=False, na=False)
        )
    )) & ( # check sufficient quantity
        ((sugar["Quantity"] >= 7.054792) & (sugar["Unit"].str.contains("oz"))) | # handle ounces case
        ((sugar["Quantity"] >= 0.440925) & (sugar["Unit"].str.contains("lb"))) # handle pounds case
    )
]
# get minimum price
sugar_svmrt.sort_values(ascending=True,by="Price").head(1)

,Brand,Price,Name,Quantity,Unit
4,C & H SUGAR,3.99,C&H® Pure Cane Granulated White Sugar 32 oz. Box,32.0,oz


### Brown Sugar

In [ ]:
query = {"query":"\n    query MerchandisedCategory($adCapabilityPackId: AdCapabilityPackId!, $networkId: String!, $siteId: String!, $limit: Int!, $offset: Int!, $taxonomyNodeId: String!, $adsScreenName: String!, $cookie: String, $sortBy: ProductSortDimension, $productQueryMatch: ProductMatchInput!, $storeId: String) {\n  heroTopAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroTop\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroTop\"\n  ) {\n    ...ad_ref_list\n  }\n  heroBottomAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroBottom\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroBottom\"\n  ) {\n    ...ad_ref_list\n  }\n  browseSubcategories(id: $taxonomyNodeId) @skip(if: true)\n  navOptionsCategories(id: $taxonomyNodeId)\n  taxonomy(id: \"Mobile/P+C\") @skip(if: true) {\n    __typename\n    ...taxonomy\n  }\n  merchandisedDisplayableObjects(\n    id: $taxonomyNodeId\n    taxonomyId: \"Mobile/P+C\"\n    positions: [$adsScreenName]\n  ) @skip(if: true) {\n    __typename\n    ... on DisplayableAd {\n      ...displayable_ad\n    }\n    ... on CategoryPreview {\n      __typename\n      ads {\n        ...ad_with_placement\n      }\n      categoryId\n      ... on ProductCategoryPreview {\n        items {\n          ...product_summary\n        }\n        prefixItems {\n          ...product_summary_with_ad_telemetry\n        }\n      }\n    }\n  }\n  products(\n    limit: $limit\n    cookie: $cookie\n    offset: $offset\n    taxonomyId: \"Mobile/P+C\"\n    sortBy: $sortBy\n    match: $productQueryMatch\n    storeId: $storeId\n  ) {\n    ...product_result\n  }\n}\n    \n    fragment ad_ref_list on AdRefList {\n  __typename\n  adRef\n  adList {\n    ...ad_with_placement\n  }\n}\n    \n\n    fragment ad_with_placement on AdWithPlacement {\n  __typename\n  position\n  ad {\n    __typename\n    ...ad_with_telemetry\n  }\n  placementId\n}\n    \n\n    fragment ad_with_telemetry on AdDTO {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  aid\n  aiid\n  height\n  pid\n  width\n  version\n  ... on CollectionAdDTO {\n    placements {\n      ...collection_ad_placement\n    }\n    ... on CarouselCollectionDTO {\n      randomizationStyle\n    }\n    title\n    subtitle\n  }\n  ... on ImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on ImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n  ... on LayoutAdDTO {\n    rows {\n      __typename\n      height\n      columns {\n        __typename\n        width\n        adRef\n      }\n    }\n  }\n  ... on RoundedImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on RoundedImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n}\n    \n\n    fragment ad_telemetry on AdTelemetry {\n  __typename\n  aid\n  aiid\n  dimensions {\n    __typename\n    key\n    value\n  }\n}\n    \n\n    fragment collection_ad_placement on PlacementDTO {\n  __typename\n  height\n  id\n  width\n  ... on LeafPlacementDTO {\n    adRef\n    ... on CarouselPlacementDTO {\n      displayDurationMs\n    }\n  }\n  ... on LayoutRowPlacementDTO {\n    placements {\n      ... on LeafPlacementDTO {\n        adRef\n        ... on CarouselPlacementDTO {\n          displayDurationMs\n        }\n      }\n    }\n  }\n}\n    \n\n    fragment ad_image_asset on ImageAsset {\n  __typename\n  id\n  densities {\n    __typename\n    swiftlyDensity\n    swiftlyDeviceClass\n    url\n  }\n  altText\n  revision\n  action {\n    __typename\n    action\n    name\n    target\n  }\n}\n    \n\n    fragment carousel_page on CarouselPage {\n  __typename\n  durationMs\n  imageAsset {\n    __typename\n    ...ad_image_asset\n  }\n}\n    \n\n    fragment taxonomy on Taxonomy {\n  allowedTypes\n  displayName\n  id\n  graph {\n    children\n    displayName\n    id\n    parents\n    renderingTemplate\n    type\n    images {\n      ...image_ref\n    }\n  }\n}\n    \n\n    fragment image_ref on ImageRef {\n  __typename\n  altText\n  caption\n  image(desiredDensity: ThreeX) {\n    ...image_file\n  }\n  images(desiredDensities: [ThreeX]) {\n    ...image_file\n  }\n  type {\n    ...image_type\n  }\n}\n    \n\n    fragment image_file on ImageFile {\n  __typename\n  density\n  device\n  uri\n}\n    \n\n    fragment image_type on ImageType {\n  __typename\n  description\n  height\n  type\n  width\n}\n    \n\n    fragment displayable_ad on DisplayableAd {\n  ad {\n    ...ad_ref_list\n  }\n}\n    \n\n    fragment product_summary on ProductSummary {\n  __typename\n  brand\n  categories\n  eligibilities {\n    __typename\n    eligibilityId\n  }\n  legacyPriceTag: fdPriceTag {\n    ...legacy_price_tag\n  }\n  id\n  images {\n    ...image_ref\n  }\n  ordinal\n  webPrice {\n    failure {\n      type\n      message\n      displayMessage\n    }\n    success {\n      basePrice\n      promoPrice\n      promoText\n      isSale\n    }\n  }\n  price {\n    ...price_result\n  }\n  shortDescription\n  tags {\n    ...product_tag\n  }\n  title\n  type\n}\n    \n\n    fragment legacy_price_tag on FamilyDollarPriceTag {\n  __typename\n  discountType\n  finalPrice {\n    ...legacy_price_detail\n  }\n  finalSavings {\n    ...legacy_savings_detail\n  }\n  priceUri\n  regularPrice {\n    ...legacy_price_detail\n  }\n  scenario\n  tags {\n    ...product_tag\n  }\n  termsAndConditions\n}\n    \n\n    fragment legacy_price_detail on PriceDetail {\n  __typename\n  buyQuantity\n  displayPrice\n  effectiveDate\n  expirationDate\n  getQuantity\n  mixAndMatchId\n  model\n  price\n  quantityLimit\n  quantityMinimum\n  soldBy\n  type\n}\n    \n\n    fragment legacy_savings_detail on SavingsDetail {\n  __typename\n  mustBuyAtLeast\n  savingsAmount\n  savingsDisplayAmount\n  savingsDisplayPercent\n  savingsPercent\n  savingsQuantity\n}\n    \n\n    fragment product_tag on Tag {\n  __typename\n  code\n  description\n  id\n  images {\n    ...image_ref\n  }\n  title\n  type\n  value\n}\n    \n\n    fragment price_result on PriceResult {\n  __typename\n  failure {\n    __typename\n    message\n    type\n  }\n  success {\n    ...price\n  }\n}\n    \n\n    fragment price on Price {\n  __typename\n  base {\n    ...price_model_properties\n  }\n  cost {\n    ...cost\n  }\n  info {\n    ...price_info\n  }\n  mixAndMatchId\n  promotion {\n    ...price_model_properties\n  }\n  savings {\n    ...savings\n  }\n  scenario\n  soldBy\n  unit\n}\n    \n\n    fragment price_model_properties on PriceModelProperties {\n  __typename\n  activeThrough {\n    ...active_through\n  }\n  amountOff {\n    ...money\n  }\n  buyQuantity\n  getQuantity\n  model\n  percentOff\n  price {\n    ...money\n  }\n  type\n}\n    \n\n    fragment active_through on ActiveThrough {\n  __typename\n  end {\n    ...active_date\n  }\n  start {\n    ...active_date\n  }\n}\n    \n\n    fragment active_date on ActiveDate {\n  __typename\n  date\n  time\n}\n    \n\n    fragment money on Money {\n  __typename\n  currency\n  denomination\n  value\n}\n    \n\n    fragment cost on Cost {\n  __typename\n  purchaseQuantity\n  totalCost {\n    ...money\n  }\n  type\n}\n    \n\n    fragment price_info on PriceInfo {\n  __typename\n  code\n  id\n  type\n  value\n}\n    \n\n    fragment savings on Savings {\n  __typename\n  percentSaved\n  savingsQuantity\n  totalSavingsAmount {\n    ...money\n  }\n  type\n}\n    \n\n    fragment product_summary_with_ad_telemetry on ProductSummaryWithAdTelemetry {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  product {\n    ...product_summary\n  }\n}\n    \n\n    fragment product_result on ProductResult {\n  __typename\n  cookie\n  count\n  fieldFacets {\n    ...field_facet\n  }\n  overlapCount\n  prefixCount\n  prefixProducts {\n    ...product_summary_with_ad_telemetry\n  }\n  products {\n    ...product_summary\n  }\n  spelling {\n    __typename\n    corrected\n    original\n  }\n}\n    \n\n    fragment field_facet on FieldFacet {\n  __typename\n  displayName\n  entries {\n    ...field_facet_entry\n  }\n  field\n}\n    \n\n    fragment field_facet_entry on FieldFacetEntry {\n  __typename\n  count\n  entry\n}\n    ","variables":{"adCapabilityPackId":"AD_CARACAL_1","adsScreenName":"ProductSearch","limit":50,"offset":0,"taxonomyNodeId":"Mobile/P+C/Product/RootCategoryId","networkId":"TSMC0.0000000002","siteId":"sm-shopperWeb","cookie":"","sortBy":"Featured","storeId":"604","productQueryMatch":{"search":{"keyword":"brown sugar"}}}}
brown = get_info(query) # get brown sugar info from savemart

In [105]:
brown

,Brand,Price,Name,Quantity,Unit
0,SUNNY SELECT,3.99,Pure Cane Light Brown Sugar 32 oz,32.00,oz
1,QUAKER,5.99,"Instant Oatmeal, Maple & Brown Sugar",NaN,None
2,SWEET BABY RAY'S,4.49,Hickory & Brown Sugar Barbecue Sauce 18 oz,18.00,oz
3,FULL CIRCLE,4.69,Light Brown Pure Sugar,NaN,None
4,HATFIELD,9.99,Savory Brown Sugar Pork Loin Filet,NaN,None
5,C & H SUGAR,4.99,C&H® Pure Cane Sugar Golden Brown Sugar 2 lb. ...,2.00,lb.
6,BUSH'S BAKED BEANS,4.49,Bush's Best Brown Sugar Hickory Baked Beans 28 oz,28.00,oz
7,BLISS,6.79,"Coffee-Mate, Oat Milk Creamer, Brown Sugar",NaN,None
8,SUNNY SELECT,3.99,Maple & Brown Sugar Instant Oatmeal,NaN,None
9,C & H SUGAR,4.99,C&H® Pure Cane Dark Brown Sugar 32 oz. Zip Pak,32.00,oz.


In [ ]:
# only want pure brown sugar
brown_svmrt = brown[
    (brown['Name'].str.contains('Pure', case=False, na=False)) & (# filter for genuine brown sugar products
    ((brown["Quantity"] >= 0.440925) & (brown["Unit"].str.contains("lb"))) | # handle pounds case
     ((brown["Quantity"] >= 7.05) & (brown["Unit"].str.contains("oz"))) # handle ounces case
)]
# check for minimum quantity
brown_svmrt.sort_values(by="Price",ascending=True).head(1)

,Brand,Price,Name,Quantity,Unit
21,C & H SUGAR,2.69,C&H® Pure Cane Golden Brown Sugar 16 oz. Box,16.0,oz.


### Eggs

In [ ]:
query = {"query":"\n    query MerchandisedCategory($adCapabilityPackId: AdCapabilityPackId!, $networkId: String!, $siteId: String!, $limit: Int!, $offset: Int!, $taxonomyNodeId: String!, $adsScreenName: String!, $cookie: String, $sortBy: ProductSortDimension, $productQueryMatch: ProductMatchInput!, $storeId: String) {\n  heroTopAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroTop\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroTop\"\n  ) {\n    ...ad_ref_list\n  }\n  heroBottomAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroBottom\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroBottom\"\n  ) {\n    ...ad_ref_list\n  }\n  browseSubcategories(id: $taxonomyNodeId) @skip(if: true)\n  navOptionsCategories(id: $taxonomyNodeId)\n  taxonomy(id: \"Mobile/P+C\") @skip(if: true) {\n    __typename\n    ...taxonomy\n  }\n  merchandisedDisplayableObjects(\n    id: $taxonomyNodeId\n    taxonomyId: \"Mobile/P+C\"\n    positions: [$adsScreenName]\n  ) @skip(if: true) {\n    __typename\n    ... on DisplayableAd {\n      ...displayable_ad\n    }\n    ... on CategoryPreview {\n      __typename\n      ads {\n        ...ad_with_placement\n      }\n      categoryId\n      ... on ProductCategoryPreview {\n        items {\n          ...product_summary\n        }\n        prefixItems {\n          ...product_summary_with_ad_telemetry\n        }\n      }\n    }\n  }\n  products(\n    limit: $limit\n    cookie: $cookie\n    offset: $offset\n    taxonomyId: \"Mobile/P+C\"\n    sortBy: $sortBy\n    match: $productQueryMatch\n    storeId: $storeId\n  ) {\n    ...product_result\n  }\n}\n    \n    fragment ad_ref_list on AdRefList {\n  __typename\n  adRef\n  adList {\n    ...ad_with_placement\n  }\n}\n    \n\n    fragment ad_with_placement on AdWithPlacement {\n  __typename\n  position\n  ad {\n    __typename\n    ...ad_with_telemetry\n  }\n  placementId\n}\n    \n\n    fragment ad_with_telemetry on AdDTO {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  aid\n  aiid\n  height\n  pid\n  width\n  version\n  ... on CollectionAdDTO {\n    placements {\n      ...collection_ad_placement\n    }\n    ... on CarouselCollectionDTO {\n      randomizationStyle\n    }\n    title\n    subtitle\n  }\n  ... on ImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on ImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n  ... on LayoutAdDTO {\n    rows {\n      __typename\n      height\n      columns {\n        __typename\n        width\n        adRef\n      }\n    }\n  }\n  ... on RoundedImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on RoundedImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n}\n    \n\n    fragment ad_telemetry on AdTelemetry {\n  __typename\n  aid\n  aiid\n  dimensions {\n    __typename\n    key\n    value\n  }\n}\n    \n\n    fragment collection_ad_placement on PlacementDTO {\n  __typename\n  height\n  id\n  width\n  ... on LeafPlacementDTO {\n    adRef\n    ... on CarouselPlacementDTO {\n      displayDurationMs\n    }\n  }\n  ... on LayoutRowPlacementDTO {\n    placements {\n      ... on LeafPlacementDTO {\n        adRef\n        ... on CarouselPlacementDTO {\n          displayDurationMs\n        }\n      }\n    }\n  }\n}\n    \n\n    fragment ad_image_asset on ImageAsset {\n  __typename\n  id\n  densities {\n    __typename\n    swiftlyDensity\n    swiftlyDeviceClass\n    url\n  }\n  altText\n  revision\n  action {\n    __typename\n    action\n    name\n    target\n  }\n}\n    \n\n    fragment carousel_page on CarouselPage {\n  __typename\n  durationMs\n  imageAsset {\n    __typename\n    ...ad_image_asset\n  }\n}\n    \n\n    fragment taxonomy on Taxonomy {\n  allowedTypes\n  displayName\n  id\n  graph {\n    children\n    displayName\n    id\n    parents\n    renderingTemplate\n    type\n    images {\n      ...image_ref\n    }\n  }\n}\n    \n\n    fragment image_ref on ImageRef {\n  __typename\n  altText\n  caption\n  image(desiredDensity: ThreeX) {\n    ...image_file\n  }\n  images(desiredDensities: [ThreeX]) {\n    ...image_file\n  }\n  type {\n    ...image_type\n  }\n}\n    \n\n    fragment image_file on ImageFile {\n  __typename\n  density\n  device\n  uri\n}\n    \n\n    fragment image_type on ImageType {\n  __typename\n  description\n  height\n  type\n  width\n}\n    \n\n    fragment displayable_ad on DisplayableAd {\n  ad {\n    ...ad_ref_list\n  }\n}\n    \n\n    fragment product_summary on ProductSummary {\n  __typename\n  brand\n  categories\n  eligibilities {\n    __typename\n    eligibilityId\n  }\n  legacyPriceTag: fdPriceTag {\n    ...legacy_price_tag\n  }\n  id\n  images {\n    ...image_ref\n  }\n  ordinal\n  webPrice {\n    failure {\n      type\n      message\n      displayMessage\n    }\n    success {\n      basePrice\n      promoPrice\n      promoText\n      isSale\n    }\n  }\n  price {\n    ...price_result\n  }\n  shortDescription\n  tags {\n    ...product_tag\n  }\n  title\n  type\n}\n    \n\n    fragment legacy_price_tag on FamilyDollarPriceTag {\n  __typename\n  discountType\n  finalPrice {\n    ...legacy_price_detail\n  }\n  finalSavings {\n    ...legacy_savings_detail\n  }\n  priceUri\n  regularPrice {\n    ...legacy_price_detail\n  }\n  scenario\n  tags {\n    ...product_tag\n  }\n  termsAndConditions\n}\n    \n\n    fragment legacy_price_detail on PriceDetail {\n  __typename\n  buyQuantity\n  displayPrice\n  effectiveDate\n  expirationDate\n  getQuantity\n  mixAndMatchId\n  model\n  price\n  quantityLimit\n  quantityMinimum\n  soldBy\n  type\n}\n    \n\n    fragment legacy_savings_detail on SavingsDetail {\n  __typename\n  mustBuyAtLeast\n  savingsAmount\n  savingsDisplayAmount\n  savingsDisplayPercent\n  savingsPercent\n  savingsQuantity\n}\n    \n\n    fragment product_tag on Tag {\n  __typename\n  code\n  description\n  id\n  images {\n    ...image_ref\n  }\n  title\n  type\n  value\n}\n    \n\n    fragment price_result on PriceResult {\n  __typename\n  failure {\n    __typename\n    message\n    type\n  }\n  success {\n    ...price\n  }\n}\n    \n\n    fragment price on Price {\n  __typename\n  base {\n    ...price_model_properties\n  }\n  cost {\n    ...cost\n  }\n  info {\n    ...price_info\n  }\n  mixAndMatchId\n  promotion {\n    ...price_model_properties\n  }\n  savings {\n    ...savings\n  }\n  scenario\n  soldBy\n  unit\n}\n    \n\n    fragment price_model_properties on PriceModelProperties {\n  __typename\n  activeThrough {\n    ...active_through\n  }\n  amountOff {\n    ...money\n  }\n  buyQuantity\n  getQuantity\n  model\n  percentOff\n  price {\n    ...money\n  }\n  type\n}\n    \n\n    fragment active_through on ActiveThrough {\n  __typename\n  end {\n    ...active_date\n  }\n  start {\n    ...active_date\n  }\n}\n    \n\n    fragment active_date on ActiveDate {\n  __typename\n  date\n  time\n}\n    \n\n    fragment money on Money {\n  __typename\n  currency\n  denomination\n  value\n}\n    \n\n    fragment cost on Cost {\n  __typename\n  purchaseQuantity\n  totalCost {\n    ...money\n  }\n  type\n}\n    \n\n    fragment price_info on PriceInfo {\n  __typename\n  code\n  id\n  type\n  value\n}\n    \n\n    fragment savings on Savings {\n  __typename\n  percentSaved\n  savingsQuantity\n  totalSavingsAmount {\n    ...money\n  }\n  type\n}\n    \n\n    fragment product_summary_with_ad_telemetry on ProductSummaryWithAdTelemetry {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  product {\n    ...product_summary\n  }\n}\n    \n\n    fragment product_result on ProductResult {\n  __typename\n  cookie\n  count\n  fieldFacets {\n    ...field_facet\n  }\n  overlapCount\n  prefixCount\n  prefixProducts {\n    ...product_summary_with_ad_telemetry\n  }\n  products {\n    ...product_summary\n  }\n  spelling {\n    __typename\n    corrected\n    original\n  }\n}\n    \n\n    fragment field_facet on FieldFacet {\n  __typename\n  displayName\n  entries {\n    ...field_facet_entry\n  }\n  field\n}\n    \n\n    fragment field_facet_entry on FieldFacetEntry {\n  __typename\n  count\n  entry\n}\n    ","variables":{"adCapabilityPackId":"AD_CARACAL_1","adsScreenName":"ProductSearch","limit":50,"offset":0,"taxonomyNodeId":"Mobile/P+C/Product/RootCategoryId","networkId":"TSMC0.0000000002","siteId":"sm-shopperWeb","cookie":"","sortBy":"Featured","storeId":"604","productQueryMatch":{"search":{"keyword":"large eggs"}}}}
eggs = get_info(query) # get egg info from savemart

Price details not available


In [117]:
eggs

,Brand,Price,Name,Quantity,Unit
0,NULAID,5.99,Large Grade AA Eggs,NaN,None
1,BAY VIEW,25.99,"Eggs, Large Grade AA",NaN,None
2,NULAID,9.59,CAGE FREE LARGE EGGS GRADE AA 18PK,18.00,pk
3,SUNNYSIDE FARMS,6.69,Extra Large Grade AA Eggs,NaN,None
4,THE HAPPY EGG COMPANY,12.99,"Happy Egg Large Brown Organic Free Range Eggs,...",18.00,count
5,ROCK ISLAND FDD,5.49,"Rock Island Brown Eggs, Large, Grade AA, Fertile",NaN,None
6,ROCK ISLAND FDD,5.49,"Rock Island Brown Eggs, Extra Large, Grade AA,...",NaN,None
7,NELLIE'S,6.49,"Nellies, Eggs, Brown, Free Range, Large",NaN,None
8,EGGLAND'S BEST,5.49,Large White Cage Free Eggs 12 ea,12.00,ea
9,EGGLAND'S BEST,7.99,Large Cage Free White Eggs 18 ea,18.00,ea


In [ ]:
# only care for entries with the word "large" in them
egg_svmrt = eggs[
    (eggs['Name'].str.contains('Large', case=False, na=False)) & 
    ((eggs['Quantity'] >=2) & ( # check for minimum quantity
        eggs["Unit"].str.contains("ct") | eggs["Unit"].str.contains("ea") | eggs["Unit"].str.contains("count")
    ))
]
# find minimum price
egg_svmrt.sort_values(by="Price",ascending=True).head(1)

,Brand,Price,Name,Quantity,Unit
17,VITAL FARMS,5.19,Large Pasture-Raised Eggs 6 ea,6.0,ea


### Vanilla Extract

In [ ]:
query = {"query":"\n    query MerchandisedCategory($adCapabilityPackId: AdCapabilityPackId!, $networkId: String!, $siteId: String!, $limit: Int!, $offset: Int!, $taxonomyNodeId: String!, $adsScreenName: String!, $cookie: String, $sortBy: ProductSortDimension, $productQueryMatch: ProductMatchInput!, $storeId: String) {\n  heroTopAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroTop\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroTop\"\n  ) {\n    ...ad_ref_list\n  }\n  heroBottomAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroBottom\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroBottom\"\n  ) {\n    ...ad_ref_list\n  }\n  browseSubcategories(id: $taxonomyNodeId) @skip(if: true)\n  navOptionsCategories(id: $taxonomyNodeId)\n  taxonomy(id: \"Mobile/P+C\") @skip(if: true) {\n    __typename\n    ...taxonomy\n  }\n  merchandisedDisplayableObjects(\n    id: $taxonomyNodeId\n    taxonomyId: \"Mobile/P+C\"\n    positions: [$adsScreenName]\n  ) @skip(if: true) {\n    __typename\n    ... on DisplayableAd {\n      ...displayable_ad\n    }\n    ... on CategoryPreview {\n      __typename\n      ads {\n        ...ad_with_placement\n      }\n      categoryId\n      ... on ProductCategoryPreview {\n        items {\n          ...product_summary\n        }\n        prefixItems {\n          ...product_summary_with_ad_telemetry\n        }\n      }\n    }\n  }\n  products(\n    limit: $limit\n    cookie: $cookie\n    offset: $offset\n    taxonomyId: \"Mobile/P+C\"\n    sortBy: $sortBy\n    match: $productQueryMatch\n    storeId: $storeId\n  ) {\n    ...product_result\n  }\n}\n    \n    fragment ad_ref_list on AdRefList {\n  __typename\n  adRef\n  adList {\n    ...ad_with_placement\n  }\n}\n    \n\n    fragment ad_with_placement on AdWithPlacement {\n  __typename\n  position\n  ad {\n    __typename\n    ...ad_with_telemetry\n  }\n  placementId\n}\n    \n\n    fragment ad_with_telemetry on AdDTO {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  aid\n  aiid\n  height\n  pid\n  width\n  version\n  ... on CollectionAdDTO {\n    placements {\n      ...collection_ad_placement\n    }\n    ... on CarouselCollectionDTO {\n      randomizationStyle\n    }\n    title\n    subtitle\n  }\n  ... on ImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on ImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n  ... on LayoutAdDTO {\n    rows {\n      __typename\n      height\n      columns {\n        __typename\n        width\n        adRef\n      }\n    }\n  }\n  ... on RoundedImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on RoundedImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n}\n    \n\n    fragment ad_telemetry on AdTelemetry {\n  __typename\n  aid\n  aiid\n  dimensions {\n    __typename\n    key\n    value\n  }\n}\n    \n\n    fragment collection_ad_placement on PlacementDTO {\n  __typename\n  height\n  id\n  width\n  ... on LeafPlacementDTO {\n    adRef\n    ... on CarouselPlacementDTO {\n      displayDurationMs\n    }\n  }\n  ... on LayoutRowPlacementDTO {\n    placements {\n      ... on LeafPlacementDTO {\n        adRef\n        ... on CarouselPlacementDTO {\n          displayDurationMs\n        }\n      }\n    }\n  }\n}\n    \n\n    fragment ad_image_asset on ImageAsset {\n  __typename\n  id\n  densities {\n    __typename\n    swiftlyDensity\n    swiftlyDeviceClass\n    url\n  }\n  altText\n  revision\n  action {\n    __typename\n    action\n    name\n    target\n  }\n}\n    \n\n    fragment carousel_page on CarouselPage {\n  __typename\n  durationMs\n  imageAsset {\n    __typename\n    ...ad_image_asset\n  }\n}\n    \n\n    fragment taxonomy on Taxonomy {\n  allowedTypes\n  displayName\n  id\n  graph {\n    children\n    displayName\n    id\n    parents\n    renderingTemplate\n    type\n    images {\n      ...image_ref\n    }\n  }\n}\n    \n\n    fragment image_ref on ImageRef {\n  __typename\n  altText\n  caption\n  image(desiredDensity: ThreeX) {\n    ...image_file\n  }\n  images(desiredDensities: [ThreeX]) {\n    ...image_file\n  }\n  type {\n    ...image_type\n  }\n}\n    \n\n    fragment image_file on ImageFile {\n  __typename\n  density\n  device\n  uri\n}\n    \n\n    fragment image_type on ImageType {\n  __typename\n  description\n  height\n  type\n  width\n}\n    \n\n    fragment displayable_ad on DisplayableAd {\n  ad {\n    ...ad_ref_list\n  }\n}\n    \n\n    fragment product_summary on ProductSummary {\n  __typename\n  brand\n  categories\n  eligibilities {\n    __typename\n    eligibilityId\n  }\n  legacyPriceTag: fdPriceTag {\n    ...legacy_price_tag\n  }\n  id\n  images {\n    ...image_ref\n  }\n  ordinal\n  webPrice {\n    failure {\n      type\n      message\n      displayMessage\n    }\n    success {\n      basePrice\n      promoPrice\n      promoText\n      isSale\n    }\n  }\n  price {\n    ...price_result\n  }\n  shortDescription\n  tags {\n    ...product_tag\n  }\n  title\n  type\n}\n    \n\n    fragment legacy_price_tag on FamilyDollarPriceTag {\n  __typename\n  discountType\n  finalPrice {\n    ...legacy_price_detail\n  }\n  finalSavings {\n    ...legacy_savings_detail\n  }\n  priceUri\n  regularPrice {\n    ...legacy_price_detail\n  }\n  scenario\n  tags {\n    ...product_tag\n  }\n  termsAndConditions\n}\n    \n\n    fragment legacy_price_detail on PriceDetail {\n  __typename\n  buyQuantity\n  displayPrice\n  effectiveDate\n  expirationDate\n  getQuantity\n  mixAndMatchId\n  model\n  price\n  quantityLimit\n  quantityMinimum\n  soldBy\n  type\n}\n    \n\n    fragment legacy_savings_detail on SavingsDetail {\n  __typename\n  mustBuyAtLeast\n  savingsAmount\n  savingsDisplayAmount\n  savingsDisplayPercent\n  savingsPercent\n  savingsQuantity\n}\n    \n\n    fragment product_tag on Tag {\n  __typename\n  code\n  description\n  id\n  images {\n    ...image_ref\n  }\n  title\n  type\n  value\n}\n    \n\n    fragment price_result on PriceResult {\n  __typename\n  failure {\n    __typename\n    message\n    type\n  }\n  success {\n    ...price\n  }\n}\n    \n\n    fragment price on Price {\n  __typename\n  base {\n    ...price_model_properties\n  }\n  cost {\n    ...cost\n  }\n  info {\n    ...price_info\n  }\n  mixAndMatchId\n  promotion {\n    ...price_model_properties\n  }\n  savings {\n    ...savings\n  }\n  scenario\n  soldBy\n  unit\n}\n    \n\n    fragment price_model_properties on PriceModelProperties {\n  __typename\n  activeThrough {\n    ...active_through\n  }\n  amountOff {\n    ...money\n  }\n  buyQuantity\n  getQuantity\n  model\n  percentOff\n  price {\n    ...money\n  }\n  type\n}\n    \n\n    fragment active_through on ActiveThrough {\n  __typename\n  end {\n    ...active_date\n  }\n  start {\n    ...active_date\n  }\n}\n    \n\n    fragment active_date on ActiveDate {\n  __typename\n  date\n  time\n}\n    \n\n    fragment money on Money {\n  __typename\n  currency\n  denomination\n  value\n}\n    \n\n    fragment cost on Cost {\n  __typename\n  purchaseQuantity\n  totalCost {\n    ...money\n  }\n  type\n}\n    \n\n    fragment price_info on PriceInfo {\n  __typename\n  code\n  id\n  type\n  value\n}\n    \n\n    fragment savings on Savings {\n  __typename\n  percentSaved\n  savingsQuantity\n  totalSavingsAmount {\n    ...money\n  }\n  type\n}\n    \n\n    fragment product_summary_with_ad_telemetry on ProductSummaryWithAdTelemetry {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  product {\n    ...product_summary\n  }\n}\n    \n\n    fragment product_result on ProductResult {\n  __typename\n  cookie\n  count\n  fieldFacets {\n    ...field_facet\n  }\n  overlapCount\n  prefixCount\n  prefixProducts {\n    ...product_summary_with_ad_telemetry\n  }\n  products {\n    ...product_summary\n  }\n  spelling {\n    __typename\n    corrected\n    original\n  }\n}\n    \n\n    fragment field_facet on FieldFacet {\n  __typename\n  displayName\n  entries {\n    ...field_facet_entry\n  }\n  field\n}\n    \n\n    fragment field_facet_entry on FieldFacetEntry {\n  __typename\n  count\n  entry\n}\n    ","variables":{"adCapabilityPackId":"AD_CARACAL_1","adsScreenName":"ProductSearch","limit":50,"offset":0,"taxonomyNodeId":"Mobile/P+C/Product/RootCategoryId","networkId":"TSMC0.0000000002","siteId":"sm-shopperWeb","cookie":"","sortBy":"Featured","storeId":"604","productQueryMatch":{"search":{"keyword":"vanilla extract"}}}}
vanilla = get_info(query) # get vanilla extract info from savemart

In [124]:
vanilla

,Brand,Price,Name,Quantity,Unit
0,SUNNY SELECT,7.99,Pure Vanilla Extract,NaN,None
1,MC CORMICK,6.99,McCormick® Pure Vanilla Extract 1 fl. oz. Box,1.0,fl. oz.
2,MC CORMICK,9.99,"McCormick® Pure Vanilla Extract, 2 fl oz",2.0,fl oz
3,MC CORMICK,17.99,"McCormick® Pure Vanilla Extract, 4 fl oz",4.0,fl oz
4,MORTON & BASSETT SPICE,15.49,Morton & Bassett Vanilla Extract 2 oz,2.0,oz
5,MORTON & BASSETT SPICE,26.49,Morton & Bassett Vanilla Extract 4 oz,4.0,oz
6,SIMPLY ORGANIC,22.99,Pure Madagascar Vanilla Extract 4 fl oz,4.0,fl oz
7,MC CORMICK,10.99,Mccormick Small Batch Pure Vanilla Extract 2 f...,2.0,fl oz
8,MC CORMICK,12.99,McCormick Gourmet™ Organic Premium Pure Vanill...,2.0,fl. oz.
9,OLIPOP SODA,2.79,Olipop Classic Root Beer Sparkling Tonic 12 fl oz,12.0,fl oz


In [ ]:
# filter for relevant products
vanilla_svmrt = vanilla[
    vanilla['Name'].str.contains('Extract', case=False, na=False) & # keyword extract
    (vanilla["Quantity"] >= 2*.2) & vanilla["Unit"].str.contains("fl") # mainly want the extract/fluid
]
vanilla_svmrt.sort_values(by="Price",ascending=True).head(1)

,Brand,Price,Name,Quantity,Unit
1,MC CORMICK,6.99,McCormick® Pure Vanilla Extract 1 fl. oz. Box,1.0,fl. oz.


### Baking Soda

In [ ]:
query = {"query":"\n    query MerchandisedCategory($adCapabilityPackId: AdCapabilityPackId!, $networkId: String!, $siteId: String!, $limit: Int!, $offset: Int!, $taxonomyNodeId: String!, $adsScreenName: String!, $cookie: String, $sortBy: ProductSortDimension, $productQueryMatch: ProductMatchInput!, $storeId: String) {\n  heroTopAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroTop\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroTop\"\n  ) {\n    ...ad_ref_list\n  }\n  heroBottomAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroBottom\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroBottom\"\n  ) {\n    ...ad_ref_list\n  }\n  browseSubcategories(id: $taxonomyNodeId) @skip(if: true)\n  navOptionsCategories(id: $taxonomyNodeId)\n  taxonomy(id: \"Mobile/P+C\") @skip(if: true) {\n    __typename\n    ...taxonomy\n  }\n  merchandisedDisplayableObjects(\n    id: $taxonomyNodeId\n    taxonomyId: \"Mobile/P+C\"\n    positions: [$adsScreenName]\n  ) @skip(if: true) {\n    __typename\n    ... on DisplayableAd {\n      ...displayable_ad\n    }\n    ... on CategoryPreview {\n      __typename\n      ads {\n        ...ad_with_placement\n      }\n      categoryId\n      ... on ProductCategoryPreview {\n        items {\n          ...product_summary\n        }\n        prefixItems {\n          ...product_summary_with_ad_telemetry\n        }\n      }\n    }\n  }\n  products(\n    limit: $limit\n    cookie: $cookie\n    offset: $offset\n    taxonomyId: \"Mobile/P+C\"\n    sortBy: $sortBy\n    match: $productQueryMatch\n    storeId: $storeId\n  ) {\n    ...product_result\n  }\n}\n    \n    fragment ad_ref_list on AdRefList {\n  __typename\n  adRef\n  adList {\n    ...ad_with_placement\n  }\n}\n    \n\n    fragment ad_with_placement on AdWithPlacement {\n  __typename\n  position\n  ad {\n    __typename\n    ...ad_with_telemetry\n  }\n  placementId\n}\n    \n\n    fragment ad_with_telemetry on AdDTO {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  aid\n  aiid\n  height\n  pid\n  width\n  version\n  ... on CollectionAdDTO {\n    placements {\n      ...collection_ad_placement\n    }\n    ... on CarouselCollectionDTO {\n      randomizationStyle\n    }\n    title\n    subtitle\n  }\n  ... on ImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on ImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n  ... on LayoutAdDTO {\n    rows {\n      __typename\n      height\n      columns {\n        __typename\n        width\n        adRef\n      }\n    }\n  }\n  ... on RoundedImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on RoundedImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n}\n    \n\n    fragment ad_telemetry on AdTelemetry {\n  __typename\n  aid\n  aiid\n  dimensions {\n    __typename\n    key\n    value\n  }\n}\n    \n\n    fragment collection_ad_placement on PlacementDTO {\n  __typename\n  height\n  id\n  width\n  ... on LeafPlacementDTO {\n    adRef\n    ... on CarouselPlacementDTO {\n      displayDurationMs\n    }\n  }\n  ... on LayoutRowPlacementDTO {\n    placements {\n      ... on LeafPlacementDTO {\n        adRef\n        ... on CarouselPlacementDTO {\n          displayDurationMs\n        }\n      }\n    }\n  }\n}\n    \n\n    fragment ad_image_asset on ImageAsset {\n  __typename\n  id\n  densities {\n    __typename\n    swiftlyDensity\n    swiftlyDeviceClass\n    url\n  }\n  altText\n  revision\n  action {\n    __typename\n    action\n    name\n    target\n  }\n}\n    \n\n    fragment carousel_page on CarouselPage {\n  __typename\n  durationMs\n  imageAsset {\n    __typename\n    ...ad_image_asset\n  }\n}\n    \n\n    fragment taxonomy on Taxonomy {\n  allowedTypes\n  displayName\n  id\n  graph {\n    children\n    displayName\n    id\n    parents\n    renderingTemplate\n    type\n    images {\n      ...image_ref\n    }\n  }\n}\n    \n\n    fragment image_ref on ImageRef {\n  __typename\n  altText\n  caption\n  image(desiredDensity: ThreeX) {\n    ...image_file\n  }\n  images(desiredDensities: [ThreeX]) {\n    ...image_file\n  }\n  type {\n    ...image_type\n  }\n}\n    \n\n    fragment image_file on ImageFile {\n  __typename\n  density\n  device\n  uri\n}\n    \n\n    fragment image_type on ImageType {\n  __typename\n  description\n  height\n  type\n  width\n}\n    \n\n    fragment displayable_ad on DisplayableAd {\n  ad {\n    ...ad_ref_list\n  }\n}\n    \n\n    fragment product_summary on ProductSummary {\n  __typename\n  brand\n  categories\n  eligibilities {\n    __typename\n    eligibilityId\n  }\n  legacyPriceTag: fdPriceTag {\n    ...legacy_price_tag\n  }\n  id\n  images {\n    ...image_ref\n  }\n  ordinal\n  webPrice {\n    failure {\n      type\n      message\n      displayMessage\n    }\n    success {\n      basePrice\n      promoPrice\n      promoText\n      isSale\n    }\n  }\n  price {\n    ...price_result\n  }\n  shortDescription\n  tags {\n    ...product_tag\n  }\n  title\n  type\n}\n    \n\n    fragment legacy_price_tag on FamilyDollarPriceTag {\n  __typename\n  discountType\n  finalPrice {\n    ...legacy_price_detail\n  }\n  finalSavings {\n    ...legacy_savings_detail\n  }\n  priceUri\n  regularPrice {\n    ...legacy_price_detail\n  }\n  scenario\n  tags {\n    ...product_tag\n  }\n  termsAndConditions\n}\n    \n\n    fragment legacy_price_detail on PriceDetail {\n  __typename\n  buyQuantity\n  displayPrice\n  effectiveDate\n  expirationDate\n  getQuantity\n  mixAndMatchId\n  model\n  price\n  quantityLimit\n  quantityMinimum\n  soldBy\n  type\n}\n    \n\n    fragment legacy_savings_detail on SavingsDetail {\n  __typename\n  mustBuyAtLeast\n  savingsAmount\n  savingsDisplayAmount\n  savingsDisplayPercent\n  savingsPercent\n  savingsQuantity\n}\n    \n\n    fragment product_tag on Tag {\n  __typename\n  code\n  description\n  id\n  images {\n    ...image_ref\n  }\n  title\n  type\n  value\n}\n    \n\n    fragment price_result on PriceResult {\n  __typename\n  failure {\n    __typename\n    message\n    type\n  }\n  success {\n    ...price\n  }\n}\n    \n\n    fragment price on Price {\n  __typename\n  base {\n    ...price_model_properties\n  }\n  cost {\n    ...cost\n  }\n  info {\n    ...price_info\n  }\n  mixAndMatchId\n  promotion {\n    ...price_model_properties\n  }\n  savings {\n    ...savings\n  }\n  scenario\n  soldBy\n  unit\n}\n    \n\n    fragment price_model_properties on PriceModelProperties {\n  __typename\n  activeThrough {\n    ...active_through\n  }\n  amountOff {\n    ...money\n  }\n  buyQuantity\n  getQuantity\n  model\n  percentOff\n  price {\n    ...money\n  }\n  type\n}\n    \n\n    fragment active_through on ActiveThrough {\n  __typename\n  end {\n    ...active_date\n  }\n  start {\n    ...active_date\n  }\n}\n    \n\n    fragment active_date on ActiveDate {\n  __typename\n  date\n  time\n}\n    \n\n    fragment money on Money {\n  __typename\n  currency\n  denomination\n  value\n}\n    \n\n    fragment cost on Cost {\n  __typename\n  purchaseQuantity\n  totalCost {\n    ...money\n  }\n  type\n}\n    \n\n    fragment price_info on PriceInfo {\n  __typename\n  code\n  id\n  type\n  value\n}\n    \n\n    fragment savings on Savings {\n  __typename\n  percentSaved\n  savingsQuantity\n  totalSavingsAmount {\n    ...money\n  }\n  type\n}\n    \n\n    fragment product_summary_with_ad_telemetry on ProductSummaryWithAdTelemetry {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  product {\n    ...product_summary\n  }\n}\n    \n\n    fragment product_result on ProductResult {\n  __typename\n  cookie\n  count\n  fieldFacets {\n    ...field_facet\n  }\n  overlapCount\n  prefixCount\n  prefixProducts {\n    ...product_summary_with_ad_telemetry\n  }\n  products {\n    ...product_summary\n  }\n  spelling {\n    __typename\n    corrected\n    original\n  }\n}\n    \n\n    fragment field_facet on FieldFacet {\n  __typename\n  displayName\n  entries {\n    ...field_facet_entry\n  }\n  field\n}\n    \n\n    fragment field_facet_entry on FieldFacetEntry {\n  __typename\n  count\n  entry\n}\n    ","variables":{"adCapabilityPackId":"AD_CARACAL_1","adsScreenName":"ProductSearch","limit":50,"offset":0,"taxonomyNodeId":"Mobile/P+C/Product/RootCategoryId","networkId":"TSMC0.0000000002","siteId":"sm-shopperWeb","cookie":"","sortBy":"Featured","storeId":"604","productQueryMatch":{"search":{"keyword":"baking soda"}}}}
soda = get_info(query) # get baking soda info from savemart

Price details not available


In [128]:
soda

,Brand,Price,Name,Quantity,Unit
0,SUNNY SELECT,1.49,Baking Soda 16 oz,16.00,oz
1,ARM & HAMMER,1.99,Pure Baking Soda 1 lb,1.00,lb
2,ARM & HAMMER,0.99,"Baking Soda, Pure",NaN,None
3,PAWS,12.49,"Clumping Cat Litter, Fragrance Free With Bakin...",NaN,None
4,ARM & HAMMER,4.59,With Activated Baking Soda Cat Litter Deodoriz...,20.00,oz
5,CREST ORAL CARE,2.89,"Crest Cavity & Tartar Protection Toothpaste, W...",4.20,oz
6,COLGATE ORAL CARE,2.99,Colgate Sparkling White Gel Baking Soda Antica...,4.00,oz
7,ARM & HAMMER,3.49,Carpet Odor Eliminator 30 oz,30.00,oz
8,CREST ORAL CARE,2.99,Crest Fluoride Anticavity Fresh Mint Toothpast...,5.70,oz
9,MC CORMICK,5.99,McCormick® Cream of Tartar 1.5 oz. Shaker,1.50,oz.


In [ ]:
# filter for relevant products
soda_svmrt = soda[
    soda['Name'].str.contains('Baking Soda', case=False, na=False) &  # contains keyword baking soda
    (((soda["Quantity"] > .2) & soda["Unit"].str.contains("oz")) | # handle ounces case
    ((soda["Quantity"] > .0125) & soda["Unit"].str.contains("lb"))) # handle lb case
]
# check for minimum price
soda_svmrt.sort_values(ascending=True,by="Price").head(1)

,Brand,Price,Name,Quantity,Unit
0,SUNNY SELECT,1.49,Baking Soda 16 oz,16.0,oz


### All-Purpose Flour

In [ ]:
query = {"query":"\n    query MerchandisedCategory($adCapabilityPackId: AdCapabilityPackId!, $networkId: String!, $siteId: String!, $limit: Int!, $offset: Int!, $taxonomyNodeId: String!, $adsScreenName: String!, $cookie: String, $sortBy: ProductSortDimension, $productQueryMatch: ProductMatchInput!, $storeId: String) {\n  heroTopAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroTop\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroTop\"\n  ) {\n    ...ad_ref_list\n  }\n  heroBottomAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroBottom\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroBottom\"\n  ) {\n    ...ad_ref_list\n  }\n  browseSubcategories(id: $taxonomyNodeId) @skip(if: true)\n  navOptionsCategories(id: $taxonomyNodeId)\n  taxonomy(id: \"Mobile/P+C\") @skip(if: true) {\n    __typename\n    ...taxonomy\n  }\n  merchandisedDisplayableObjects(\n    id: $taxonomyNodeId\n    taxonomyId: \"Mobile/P+C\"\n    positions: [$adsScreenName]\n  ) @skip(if: true) {\n    __typename\n    ... on DisplayableAd {\n      ...displayable_ad\n    }\n    ... on CategoryPreview {\n      __typename\n      ads {\n        ...ad_with_placement\n      }\n      categoryId\n      ... on ProductCategoryPreview {\n        items {\n          ...product_summary\n        }\n        prefixItems {\n          ...product_summary_with_ad_telemetry\n        }\n      }\n    }\n  }\n  products(\n    limit: $limit\n    cookie: $cookie\n    offset: $offset\n    taxonomyId: \"Mobile/P+C\"\n    sortBy: $sortBy\n    match: $productQueryMatch\n    storeId: $storeId\n  ) {\n    ...product_result\n  }\n}\n    \n    fragment ad_ref_list on AdRefList {\n  __typename\n  adRef\n  adList {\n    ...ad_with_placement\n  }\n}\n    \n\n    fragment ad_with_placement on AdWithPlacement {\n  __typename\n  position\n  ad {\n    __typename\n    ...ad_with_telemetry\n  }\n  placementId\n}\n    \n\n    fragment ad_with_telemetry on AdDTO {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  aid\n  aiid\n  height\n  pid\n  width\n  version\n  ... on CollectionAdDTO {\n    placements {\n      ...collection_ad_placement\n    }\n    ... on CarouselCollectionDTO {\n      randomizationStyle\n    }\n    title\n    subtitle\n  }\n  ... on ImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on ImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n  ... on LayoutAdDTO {\n    rows {\n      __typename\n      height\n      columns {\n        __typename\n        width\n        adRef\n      }\n    }\n  }\n  ... on RoundedImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on RoundedImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n}\n    \n\n    fragment ad_telemetry on AdTelemetry {\n  __typename\n  aid\n  aiid\n  dimensions {\n    __typename\n    key\n    value\n  }\n}\n    \n\n    fragment collection_ad_placement on PlacementDTO {\n  __typename\n  height\n  id\n  width\n  ... on LeafPlacementDTO {\n    adRef\n    ... on CarouselPlacementDTO {\n      displayDurationMs\n    }\n  }\n  ... on LayoutRowPlacementDTO {\n    placements {\n      ... on LeafPlacementDTO {\n        adRef\n        ... on CarouselPlacementDTO {\n          displayDurationMs\n        }\n      }\n    }\n  }\n}\n    \n\n    fragment ad_image_asset on ImageAsset {\n  __typename\n  id\n  densities {\n    __typename\n    swiftlyDensity\n    swiftlyDeviceClass\n    url\n  }\n  altText\n  revision\n  action {\n    __typename\n    action\n    name\n    target\n  }\n}\n    \n\n    fragment carousel_page on CarouselPage {\n  __typename\n  durationMs\n  imageAsset {\n    __typename\n    ...ad_image_asset\n  }\n}\n    \n\n    fragment taxonomy on Taxonomy {\n  allowedTypes\n  displayName\n  id\n  graph {\n    children\n    displayName\n    id\n    parents\n    renderingTemplate\n    type\n    images {\n      ...image_ref\n    }\n  }\n}\n    \n\n    fragment image_ref on ImageRef {\n  __typename\n  altText\n  caption\n  image(desiredDensity: ThreeX) {\n    ...image_file\n  }\n  images(desiredDensities: [ThreeX]) {\n    ...image_file\n  }\n  type {\n    ...image_type\n  }\n}\n    \n\n    fragment image_file on ImageFile {\n  __typename\n  density\n  device\n  uri\n}\n    \n\n    fragment image_type on ImageType {\n  __typename\n  description\n  height\n  type\n  width\n}\n    \n\n    fragment displayable_ad on DisplayableAd {\n  ad {\n    ...ad_ref_list\n  }\n}\n    \n\n    fragment product_summary on ProductSummary {\n  __typename\n  brand\n  categories\n  eligibilities {\n    __typename\n    eligibilityId\n  }\n  legacyPriceTag: fdPriceTag {\n    ...legacy_price_tag\n  }\n  id\n  images {\n    ...image_ref\n  }\n  ordinal\n  webPrice {\n    failure {\n      type\n      message\n      displayMessage\n    }\n    success {\n      basePrice\n      promoPrice\n      promoText\n      isSale\n    }\n  }\n  price {\n    ...price_result\n  }\n  shortDescription\n  tags {\n    ...product_tag\n  }\n  title\n  type\n}\n    \n\n    fragment legacy_price_tag on FamilyDollarPriceTag {\n  __typename\n  discountType\n  finalPrice {\n    ...legacy_price_detail\n  }\n  finalSavings {\n    ...legacy_savings_detail\n  }\n  priceUri\n  regularPrice {\n    ...legacy_price_detail\n  }\n  scenario\n  tags {\n    ...product_tag\n  }\n  termsAndConditions\n}\n    \n\n    fragment legacy_price_detail on PriceDetail {\n  __typename\n  buyQuantity\n  displayPrice\n  effectiveDate\n  expirationDate\n  getQuantity\n  mixAndMatchId\n  model\n  price\n  quantityLimit\n  quantityMinimum\n  soldBy\n  type\n}\n    \n\n    fragment legacy_savings_detail on SavingsDetail {\n  __typename\n  mustBuyAtLeast\n  savingsAmount\n  savingsDisplayAmount\n  savingsDisplayPercent\n  savingsPercent\n  savingsQuantity\n}\n    \n\n    fragment product_tag on Tag {\n  __typename\n  code\n  description\n  id\n  images {\n    ...image_ref\n  }\n  title\n  type\n  value\n}\n    \n\n    fragment price_result on PriceResult {\n  __typename\n  failure {\n    __typename\n    message\n    type\n  }\n  success {\n    ...price\n  }\n}\n    \n\n    fragment price on Price {\n  __typename\n  base {\n    ...price_model_properties\n  }\n  cost {\n    ...cost\n  }\n  info {\n    ...price_info\n  }\n  mixAndMatchId\n  promotion {\n    ...price_model_properties\n  }\n  savings {\n    ...savings\n  }\n  scenario\n  soldBy\n  unit\n}\n    \n\n    fragment price_model_properties on PriceModelProperties {\n  __typename\n  activeThrough {\n    ...active_through\n  }\n  amountOff {\n    ...money\n  }\n  buyQuantity\n  getQuantity\n  model\n  percentOff\n  price {\n    ...money\n  }\n  type\n}\n    \n\n    fragment active_through on ActiveThrough {\n  __typename\n  end {\n    ...active_date\n  }\n  start {\n    ...active_date\n  }\n}\n    \n\n    fragment active_date on ActiveDate {\n  __typename\n  date\n  time\n}\n    \n\n    fragment money on Money {\n  __typename\n  currency\n  denomination\n  value\n}\n    \n\n    fragment cost on Cost {\n  __typename\n  purchaseQuantity\n  totalCost {\n    ...money\n  }\n  type\n}\n    \n\n    fragment price_info on PriceInfo {\n  __typename\n  code\n  id\n  type\n  value\n}\n    \n\n    fragment savings on Savings {\n  __typename\n  percentSaved\n  savingsQuantity\n  totalSavingsAmount {\n    ...money\n  }\n  type\n}\n    \n\n    fragment product_summary_with_ad_telemetry on ProductSummaryWithAdTelemetry {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  product {\n    ...product_summary\n  }\n}\n    \n\n    fragment product_result on ProductResult {\n  __typename\n  cookie\n  count\n  fieldFacets {\n    ...field_facet\n  }\n  overlapCount\n  prefixCount\n  prefixProducts {\n    ...product_summary_with_ad_telemetry\n  }\n  products {\n    ...product_summary\n  }\n  spelling {\n    __typename\n    corrected\n    original\n  }\n}\n    \n\n    fragment field_facet on FieldFacet {\n  __typename\n  displayName\n  entries {\n    ...field_facet_entry\n  }\n  field\n}\n    \n\n    fragment field_facet_entry on FieldFacetEntry {\n  __typename\n  count\n  entry\n}\n    ","variables":{"adCapabilityPackId":"AD_CARACAL_1","adsScreenName":"ProductSearch","limit":50,"offset":0,"taxonomyNodeId":"Mobile/P+C/Product/RootCategoryId","networkId":"TSMC0.0000000002","siteId":"sm-shopperWeb","cookie":"","sortBy":"Featured","storeId":"604","productQueryMatch":{"search":{"keyword":"all-purpose flour"}}}}
flour = get_info(query) # get all-purpose flour info from savemart

In [132]:
flour

,Brand,Price,Name,Quantity,Unit
0,GOLD MEDAL,6.49,All Purpose Flour 5 lb,5.00,lb
1,KING ARTHUR MIX,5.69,Organic All-Purpose Flour,NaN,None
2,GOLD MEDAL,6.49,All Purpose Flour 4.25 lb,4.25,lb
3,SUNNY SELECT,17.99,"All Purpose Flour, 25lb",25.00,lb
4,GOLD MEDAL,6.49,Unbleached All Purpose Flour 5 lb,5.00,lb
5,KING ARTHUR MIX,9.49,Gluten Free All Purpose Flour,NaN,None
6,KING ARTHUR MIX,8.69,King Arthur Baking Company Unbleached All-Purp...,5.00,lb
7,GOLD MEDAL,4.99,"Wondra Quick Mixing All Purpose Flour, 13.5 oz",13.50,oz
8,FULL CIRCLE,7.49,Market Unbleached Organic All-Purpose Flour 5 lb,5.00,lb
9,BOB'S RED MILL,6.69,Gluten free All-Purpose Baking Flour 22 oz,22.00,oz


In [ ]:
# filter for entries w/ the phrase "All-Purpose Flour"
flour_svmrt = flour[
    (flour['Name'].str.contains('Purpose Flour', case=False, na=False)) & 
    ((flour["Unit"].str.contains("lb") & (flour["Quantity"] >= 3*0.275578)) | # check for minimum ounces
     (flour["Unit"].str.contains("oz") & (flour["Quantity"] >= 3*4.40925))) # check for minimum pounds
]
# check for minimum price
flour_svmrt.sort_values(ascending=True,by="Price").head(1)
# top choice might not be enough flour
# 3 cups of flour > 16 oz (1 lb)
# top three are tied. Let's just take the first one.

,Brand,Price,Name,Quantity,Unit
0,GOLD MEDAL,6.49,All Purpose Flour 5 lb,5.0,lb


### Chocolate Chips
12 oz = 2 cups

In [ ]:
query = {"query":"\n    query MerchandisedCategory($adCapabilityPackId: AdCapabilityPackId!, $networkId: String!, $siteId: String!, $limit: Int!, $offset: Int!, $taxonomyNodeId: String!, $adsScreenName: String!, $cookie: String, $sortBy: ProductSortDimension, $productQueryMatch: ProductMatchInput!, $storeId: String) {\n  heroTopAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroTop\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroTop\"\n  ) {\n    ...ad_ref_list\n  }\n  heroBottomAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroBottom\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroBottom\"\n  ) {\n    ...ad_ref_list\n  }\n  browseSubcategories(id: $taxonomyNodeId) @skip(if: true)\n  navOptionsCategories(id: $taxonomyNodeId)\n  taxonomy(id: \"Mobile/P+C\") @skip(if: true) {\n    __typename\n    ...taxonomy\n  }\n  merchandisedDisplayableObjects(\n    id: $taxonomyNodeId\n    taxonomyId: \"Mobile/P+C\"\n    positions: [$adsScreenName]\n  ) @skip(if: true) {\n    __typename\n    ... on DisplayableAd {\n      ...displayable_ad\n    }\n    ... on CategoryPreview {\n      __typename\n      ads {\n        ...ad_with_placement\n      }\n      categoryId\n      ... on ProductCategoryPreview {\n        items {\n          ...product_summary\n        }\n        prefixItems {\n          ...product_summary_with_ad_telemetry\n        }\n      }\n    }\n  }\n  products(\n    limit: $limit\n    cookie: $cookie\n    offset: $offset\n    taxonomyId: \"Mobile/P+C\"\n    sortBy: $sortBy\n    match: $productQueryMatch\n    storeId: $storeId\n  ) {\n    ...product_result\n  }\n}\n    \n    fragment ad_ref_list on AdRefList {\n  __typename\n  adRef\n  adList {\n    ...ad_with_placement\n  }\n}\n    \n\n    fragment ad_with_placement on AdWithPlacement {\n  __typename\n  position\n  ad {\n    __typename\n    ...ad_with_telemetry\n  }\n  placementId\n}\n    \n\n    fragment ad_with_telemetry on AdDTO {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  aid\n  aiid\n  height\n  pid\n  width\n  version\n  ... on CollectionAdDTO {\n    placements {\n      ...collection_ad_placement\n    }\n    ... on CarouselCollectionDTO {\n      randomizationStyle\n    }\n    title\n    subtitle\n  }\n  ... on ImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on ImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n  ... on LayoutAdDTO {\n    rows {\n      __typename\n      height\n      columns {\n        __typename\n        width\n        adRef\n      }\n    }\n  }\n  ... on RoundedImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on RoundedImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n}\n    \n\n    fragment ad_telemetry on AdTelemetry {\n  __typename\n  aid\n  aiid\n  dimensions {\n    __typename\n    key\n    value\n  }\n}\n    \n\n    fragment collection_ad_placement on PlacementDTO {\n  __typename\n  height\n  id\n  width\n  ... on LeafPlacementDTO {\n    adRef\n    ... on CarouselPlacementDTO {\n      displayDurationMs\n    }\n  }\n  ... on LayoutRowPlacementDTO {\n    placements {\n      ... on LeafPlacementDTO {\n        adRef\n        ... on CarouselPlacementDTO {\n          displayDurationMs\n        }\n      }\n    }\n  }\n}\n    \n\n    fragment ad_image_asset on ImageAsset {\n  __typename\n  id\n  densities {\n    __typename\n    swiftlyDensity\n    swiftlyDeviceClass\n    url\n  }\n  altText\n  revision\n  action {\n    __typename\n    action\n    name\n    target\n  }\n}\n    \n\n    fragment carousel_page on CarouselPage {\n  __typename\n  durationMs\n  imageAsset {\n    __typename\n    ...ad_image_asset\n  }\n}\n    \n\n    fragment taxonomy on Taxonomy {\n  allowedTypes\n  displayName\n  id\n  graph {\n    children\n    displayName\n    id\n    parents\n    renderingTemplate\n    type\n    images {\n      ...image_ref\n    }\n  }\n}\n    \n\n    fragment image_ref on ImageRef {\n  __typename\n  altText\n  caption\n  image(desiredDensity: ThreeX) {\n    ...image_file\n  }\n  images(desiredDensities: [ThreeX]) {\n    ...image_file\n  }\n  type {\n    ...image_type\n  }\n}\n    \n\n    fragment image_file on ImageFile {\n  __typename\n  density\n  device\n  uri\n}\n    \n\n    fragment image_type on ImageType {\n  __typename\n  description\n  height\n  type\n  width\n}\n    \n\n    fragment displayable_ad on DisplayableAd {\n  ad {\n    ...ad_ref_list\n  }\n}\n    \n\n    fragment product_summary on ProductSummary {\n  __typename\n  brand\n  categories\n  eligibilities {\n    __typename\n    eligibilityId\n  }\n  legacyPriceTag: fdPriceTag {\n    ...legacy_price_tag\n  }\n  id\n  images {\n    ...image_ref\n  }\n  ordinal\n  webPrice {\n    failure {\n      type\n      message\n      displayMessage\n    }\n    success {\n      basePrice\n      promoPrice\n      promoText\n      isSale\n    }\n  }\n  price {\n    ...price_result\n  }\n  shortDescription\n  tags {\n    ...product_tag\n  }\n  title\n  type\n}\n    \n\n    fragment legacy_price_tag on FamilyDollarPriceTag {\n  __typename\n  discountType\n  finalPrice {\n    ...legacy_price_detail\n  }\n  finalSavings {\n    ...legacy_savings_detail\n  }\n  priceUri\n  regularPrice {\n    ...legacy_price_detail\n  }\n  scenario\n  tags {\n    ...product_tag\n  }\n  termsAndConditions\n}\n    \n\n    fragment legacy_price_detail on PriceDetail {\n  __typename\n  buyQuantity\n  displayPrice\n  effectiveDate\n  expirationDate\n  getQuantity\n  mixAndMatchId\n  model\n  price\n  quantityLimit\n  quantityMinimum\n  soldBy\n  type\n}\n    \n\n    fragment legacy_savings_detail on SavingsDetail {\n  __typename\n  mustBuyAtLeast\n  savingsAmount\n  savingsDisplayAmount\n  savingsDisplayPercent\n  savingsPercent\n  savingsQuantity\n}\n    \n\n    fragment product_tag on Tag {\n  __typename\n  code\n  description\n  id\n  images {\n    ...image_ref\n  }\n  title\n  type\n  value\n}\n    \n\n    fragment price_result on PriceResult {\n  __typename\n  failure {\n    __typename\n    message\n    type\n  }\n  success {\n    ...price\n  }\n}\n    \n\n    fragment price on Price {\n  __typename\n  base {\n    ...price_model_properties\n  }\n  cost {\n    ...cost\n  }\n  info {\n    ...price_info\n  }\n  mixAndMatchId\n  promotion {\n    ...price_model_properties\n  }\n  savings {\n    ...savings\n  }\n  scenario\n  soldBy\n  unit\n}\n    \n\n    fragment price_model_properties on PriceModelProperties {\n  __typename\n  activeThrough {\n    ...active_through\n  }\n  amountOff {\n    ...money\n  }\n  buyQuantity\n  getQuantity\n  model\n  percentOff\n  price {\n    ...money\n  }\n  type\n}\n    \n\n    fragment active_through on ActiveThrough {\n  __typename\n  end {\n    ...active_date\n  }\n  start {\n    ...active_date\n  }\n}\n    \n\n    fragment active_date on ActiveDate {\n  __typename\n  date\n  time\n}\n    \n\n    fragment money on Money {\n  __typename\n  currency\n  denomination\n  value\n}\n    \n\n    fragment cost on Cost {\n  __typename\n  purchaseQuantity\n  totalCost {\n    ...money\n  }\n  type\n}\n    \n\n    fragment price_info on PriceInfo {\n  __typename\n  code\n  id\n  type\n  value\n}\n    \n\n    fragment savings on Savings {\n  __typename\n  percentSaved\n  savingsQuantity\n  totalSavingsAmount {\n    ...money\n  }\n  type\n}\n    \n\n    fragment product_summary_with_ad_telemetry on ProductSummaryWithAdTelemetry {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  product {\n    ...product_summary\n  }\n}\n    \n\n    fragment product_result on ProductResult {\n  __typename\n  cookie\n  count\n  fieldFacets {\n    ...field_facet\n  }\n  overlapCount\n  prefixCount\n  prefixProducts {\n    ...product_summary_with_ad_telemetry\n  }\n  products {\n    ...product_summary\n  }\n  spelling {\n    __typename\n    corrected\n    original\n  }\n}\n    \n\n    fragment field_facet on FieldFacet {\n  __typename\n  displayName\n  entries {\n    ...field_facet_entry\n  }\n  field\n}\n    \n\n    fragment field_facet_entry on FieldFacetEntry {\n  __typename\n  count\n  entry\n}\n    ","variables":{"adCapabilityPackId":"AD_CARACAL_1","adsScreenName":"ProductSearch","limit":50,"offset":0,"taxonomyNodeId":"Mobile/P+C/Product/RootCategoryId","networkId":"TSMC0.0000000002","siteId":"sm-shopperWeb","cookie":"","sortBy":"Featured","storeId":"604","productQueryMatch":{"search":{"keyword":"semisweet chocolate chips"}}}}
chips = get_info(query) # get info about chocolate chips

In [136]:
chips

,Brand,Price,Name,Quantity,Unit
0,GUITARD'S CHOCOLATE,6.99,Guittard Semisweet Chocolate Baking Chips 12 oz,12.0,oz
1,SWERVE.LLC,8.29,Swerve Semisweet Style Zero Sugar Chocolate Ch...,9.0,oz
2,TALENTI ITALIAN ICE CREAM,6.49,Talenti Mediterranean Mint Gelato 1 pt,NaN,None


In [ ]:
# filter for elements w/ the word chips in the name
chips_svmrt = chips[
    (chips['Name'].str.contains('Chips', case=False, na=False)) & (
        chips['Unit'].str.contains('oz', regex=True) # ensures there is enough quantity
    ) & (
        chips['Quantity'] >= 2*6
    )
]
# check for minimum price
chips_svmrt.sort_values(ascending=True,by="Price").head(1)
# contains enough chips, since it is a package of 12 ounces

,Brand,Price,Name,Quantity,Unit
0,GUITARD'S CHOCOLATE,6.99,Guittard Semisweet Chocolate Baking Chips 12 oz,12.0,oz


### Chopped Walnuts
1 cup = 4.4 oz

In [ ]:
query = {"query":"\n    query MerchandisedCategory($adCapabilityPackId: AdCapabilityPackId!, $networkId: String!, $siteId: String!, $limit: Int!, $offset: Int!, $taxonomyNodeId: String!, $adsScreenName: String!, $cookie: String, $sortBy: ProductSortDimension, $productQueryMatch: ProductMatchInput!, $storeId: String) {\n  heroTopAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroTop\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroTop\"\n  ) {\n    ...ad_ref_list\n  }\n  heroBottomAd: singleAd(\n    adCapabilityPackId: $adCapabilityPackId\n    adParams: [{key: \"position\", value: \"HeroBottom\"}, {key: \"asn\", value: $adsScreenName}, {key: \"ca\", value: $taxonomyNodeId}, {key: \"sz\", value: \"375x*\"}]\n    networkId: $networkId\n    siteId: $siteId\n    placementId: \"HeroBottom\"\n  ) {\n    ...ad_ref_list\n  }\n  browseSubcategories(id: $taxonomyNodeId) @skip(if: true)\n  navOptionsCategories(id: $taxonomyNodeId)\n  taxonomy(id: \"Mobile/P+C\") @skip(if: true) {\n    __typename\n    ...taxonomy\n  }\n  merchandisedDisplayableObjects(\n    id: $taxonomyNodeId\n    taxonomyId: \"Mobile/P+C\"\n    positions: [$adsScreenName]\n  ) @skip(if: true) {\n    __typename\n    ... on DisplayableAd {\n      ...displayable_ad\n    }\n    ... on CategoryPreview {\n      __typename\n      ads {\n        ...ad_with_placement\n      }\n      categoryId\n      ... on ProductCategoryPreview {\n        items {\n          ...product_summary\n        }\n        prefixItems {\n          ...product_summary_with_ad_telemetry\n        }\n      }\n    }\n  }\n  products(\n    limit: $limit\n    cookie: $cookie\n    offset: $offset\n    taxonomyId: \"Mobile/P+C\"\n    sortBy: $sortBy\n    match: $productQueryMatch\n    storeId: $storeId\n  ) {\n    ...product_result\n  }\n}\n    \n    fragment ad_ref_list on AdRefList {\n  __typename\n  adRef\n  adList {\n    ...ad_with_placement\n  }\n}\n    \n\n    fragment ad_with_placement on AdWithPlacement {\n  __typename\n  position\n  ad {\n    __typename\n    ...ad_with_telemetry\n  }\n  placementId\n}\n    \n\n    fragment ad_with_telemetry on AdDTO {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  aid\n  aiid\n  height\n  pid\n  width\n  version\n  ... on CollectionAdDTO {\n    placements {\n      ...collection_ad_placement\n    }\n    ... on CarouselCollectionDTO {\n      randomizationStyle\n    }\n    title\n    subtitle\n  }\n  ... on ImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on ImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n  ... on LayoutAdDTO {\n    rows {\n      __typename\n      height\n      columns {\n        __typename\n        width\n        adRef\n      }\n    }\n  }\n  ... on RoundedImageAdDTO {\n    image {\n      ...ad_image_asset\n    }\n  }\n  ... on RoundedImageCarouselAdDTO {\n    carouselPages {\n      ...carousel_page\n    }\n  }\n}\n    \n\n    fragment ad_telemetry on AdTelemetry {\n  __typename\n  aid\n  aiid\n  dimensions {\n    __typename\n    key\n    value\n  }\n}\n    \n\n    fragment collection_ad_placement on PlacementDTO {\n  __typename\n  height\n  id\n  width\n  ... on LeafPlacementDTO {\n    adRef\n    ... on CarouselPlacementDTO {\n      displayDurationMs\n    }\n  }\n  ... on LayoutRowPlacementDTO {\n    placements {\n      ... on LeafPlacementDTO {\n        adRef\n        ... on CarouselPlacementDTO {\n          displayDurationMs\n        }\n      }\n    }\n  }\n}\n    \n\n    fragment ad_image_asset on ImageAsset {\n  __typename\n  id\n  densities {\n    __typename\n    swiftlyDensity\n    swiftlyDeviceClass\n    url\n  }\n  altText\n  revision\n  action {\n    __typename\n    action\n    name\n    target\n  }\n}\n    \n\n    fragment carousel_page on CarouselPage {\n  __typename\n  durationMs\n  imageAsset {\n    __typename\n    ...ad_image_asset\n  }\n}\n    \n\n    fragment taxonomy on Taxonomy {\n  allowedTypes\n  displayName\n  id\n  graph {\n    children\n    displayName\n    id\n    parents\n    renderingTemplate\n    type\n    images {\n      ...image_ref\n    }\n  }\n}\n    \n\n    fragment image_ref on ImageRef {\n  __typename\n  altText\n  caption\n  image(desiredDensity: ThreeX) {\n    ...image_file\n  }\n  images(desiredDensities: [ThreeX]) {\n    ...image_file\n  }\n  type {\n    ...image_type\n  }\n}\n    \n\n    fragment image_file on ImageFile {\n  __typename\n  density\n  device\n  uri\n}\n    \n\n    fragment image_type on ImageType {\n  __typename\n  description\n  height\n  type\n  width\n}\n    \n\n    fragment displayable_ad on DisplayableAd {\n  ad {\n    ...ad_ref_list\n  }\n}\n    \n\n    fragment product_summary on ProductSummary {\n  __typename\n  brand\n  categories\n  eligibilities {\n    __typename\n    eligibilityId\n  }\n  legacyPriceTag: fdPriceTag {\n    ...legacy_price_tag\n  }\n  id\n  images {\n    ...image_ref\n  }\n  ordinal\n  webPrice {\n    failure {\n      type\n      message\n      displayMessage\n    }\n    success {\n      basePrice\n      promoPrice\n      promoText\n      isSale\n    }\n  }\n  price {\n    ...price_result\n  }\n  shortDescription\n  tags {\n    ...product_tag\n  }\n  title\n  type\n}\n    \n\n    fragment legacy_price_tag on FamilyDollarPriceTag {\n  __typename\n  discountType\n  finalPrice {\n    ...legacy_price_detail\n  }\n  finalSavings {\n    ...legacy_savings_detail\n  }\n  priceUri\n  regularPrice {\n    ...legacy_price_detail\n  }\n  scenario\n  tags {\n    ...product_tag\n  }\n  termsAndConditions\n}\n    \n\n    fragment legacy_price_detail on PriceDetail {\n  __typename\n  buyQuantity\n  displayPrice\n  effectiveDate\n  expirationDate\n  getQuantity\n  mixAndMatchId\n  model\n  price\n  quantityLimit\n  quantityMinimum\n  soldBy\n  type\n}\n    \n\n    fragment legacy_savings_detail on SavingsDetail {\n  __typename\n  mustBuyAtLeast\n  savingsAmount\n  savingsDisplayAmount\n  savingsDisplayPercent\n  savingsPercent\n  savingsQuantity\n}\n    \n\n    fragment product_tag on Tag {\n  __typename\n  code\n  description\n  id\n  images {\n    ...image_ref\n  }\n  title\n  type\n  value\n}\n    \n\n    fragment price_result on PriceResult {\n  __typename\n  failure {\n    __typename\n    message\n    type\n  }\n  success {\n    ...price\n  }\n}\n    \n\n    fragment price on Price {\n  __typename\n  base {\n    ...price_model_properties\n  }\n  cost {\n    ...cost\n  }\n  info {\n    ...price_info\n  }\n  mixAndMatchId\n  promotion {\n    ...price_model_properties\n  }\n  savings {\n    ...savings\n  }\n  scenario\n  soldBy\n  unit\n}\n    \n\n    fragment price_model_properties on PriceModelProperties {\n  __typename\n  activeThrough {\n    ...active_through\n  }\n  amountOff {\n    ...money\n  }\n  buyQuantity\n  getQuantity\n  model\n  percentOff\n  price {\n    ...money\n  }\n  type\n}\n    \n\n    fragment active_through on ActiveThrough {\n  __typename\n  end {\n    ...active_date\n  }\n  start {\n    ...active_date\n  }\n}\n    \n\n    fragment active_date on ActiveDate {\n  __typename\n  date\n  time\n}\n    \n\n    fragment money on Money {\n  __typename\n  currency\n  denomination\n  value\n}\n    \n\n    fragment cost on Cost {\n  __typename\n  purchaseQuantity\n  totalCost {\n    ...money\n  }\n  type\n}\n    \n\n    fragment price_info on PriceInfo {\n  __typename\n  code\n  id\n  type\n  value\n}\n    \n\n    fragment savings on Savings {\n  __typename\n  percentSaved\n  savingsQuantity\n  totalSavingsAmount {\n    ...money\n  }\n  type\n}\n    \n\n    fragment product_summary_with_ad_telemetry on ProductSummaryWithAdTelemetry {\n  __typename\n  adTelemetry {\n    ...ad_telemetry\n  }\n  product {\n    ...product_summary\n  }\n}\n    \n\n    fragment product_result on ProductResult {\n  __typename\n  cookie\n  count\n  fieldFacets {\n    ...field_facet\n  }\n  overlapCount\n  prefixCount\n  prefixProducts {\n    ...product_summary_with_ad_telemetry\n  }\n  products {\n    ...product_summary\n  }\n  spelling {\n    __typename\n    corrected\n    original\n  }\n}\n    \n\n    fragment field_facet on FieldFacet {\n  __typename\n  displayName\n  entries {\n    ...field_facet_entry\n  }\n  field\n}\n    \n\n    fragment field_facet_entry on FieldFacetEntry {\n  __typename\n  count\n  entry\n}\n    ","variables":{"adCapabilityPackId":"AD_CARACAL_1","adsScreenName":"ProductSearch","limit":50,"offset":0,"taxonomyNodeId":"Mobile/P+C/Product/RootCategoryId","networkId":"TSMC0.0000000002","siteId":"sm-shopperWeb","cookie":"","sortBy":"Featured","storeId":"604","productQueryMatch":{"search":{"keyword":"chopped walnuts"}}}}
walnuts = get_info(query) # get the product info about walnuts

In [139]:
walnuts

,Brand,Price,Name,Quantity,Unit
0,DIAMOND NUT,7.99,Diamond of California Chopped Walnuts 8 oz,8.00,oz
1,DIAMOND NUT,10.99,Diamond Chopped Walnuts 14 oz,14.00,oz
2,DIAMOND NUT,3.99,Diamond Chopped Walnuts 2.25 oz,2.25,oz
3,DSD MERCHANDERS BULK FOODS,6.19,DSD Merchandisers Chopped Walnuts 9 oz,9.00,oz
4,DIAMOND NUT,9.99,Diamond of California Chopped Pecans 8 oz,8.00,oz
5,DIAMOND NUT,4.89,Diamond Chopped Pecans 2.25 oz,2.25,oz
6,DIAMOND NUT,17.99,Diamond of California Chopped Pecans 16 oz,16.00,oz
7,DIAMOND NUT,4.79,Diamond Chopped Hazelnuts 2.25 oz,2.25,oz
8,GHIRARDELLI,5.69,Premium Caramel Turtle Brownie Mix 18.5 oz,18.50,oz
9,KELLOGG'S,8.49,All Bran Buds Cereal 22 oz,22.00,oz


In [ ]:
# filter for elements w/ the word walnuts in the name
walnuts_svmrt = walnuts[
    (walnuts['Name'].str.contains('Walnuts', case=False, na=False)) &
    (walnuts["Quantity"] >= 4.4) # all elements have same unit, so check whether has minimum qty
]
# check for minimum price
walnuts_svmrt.sort_values(ascending=True,by="Price").head(1)
# the top choice doesn't have a minimum of 4.4 oz
# so we select the next one
# contains enough chips, since it is a package of 12 ounces

,Brand,Price,Name,Quantity,Unit
3,DSD MERCHANDERS BULK FOODS,6.19,DSD Merchandisers Chopped Walnuts 9 oz,9.0,oz


### Total Price

In [ ]:
# organize into one table
total_price = pd.concat([butter_svmrt.sort_values(ascending=True,by="Price").head(1),
                         sugar_svmrt.sort_values(ascending=True,by="Price").head(1),
                         brown_svmrt.sort_values(by="Price",ascending=True).head(1),
                         egg_svmrt.sort_values(by="Price",ascending=True).head(1),
                         vanilla_svmrt.sort_values(by="Price",ascending=True).head(1),
                         soda_svmrt.sort_values(ascending=True,by="Price").head(1),
                         flour_svmrt.sort_values(ascending=True,by="Price").head(1),
                         chips_svmrt.sort_values(ascending=True,by="Price").head(1),
                         walnuts_svmrt.sort_values(ascending=True,by="Price").head(1)])

In [ ]:
sum(total_price["Price"]) # calculate total cost

46.010000000000005

## Safeway
Get products from Safeway

In [ ]:
# function to get the search results form safeway
def safeway_search(ingredient:str):
    # initialize cookies
    s = requests.Session()
    s.cookies
    url = 'https://www.safeway.com/abs/pub/xapi/search/products'
    i = 0 # start value
    rows = 30 # results per query
    all_res = [] # list to store all results
    # loop through the search page results
    # and exit when no results are returned
    while True:  
        # search parameters
        params = {
            'q': ingredient,
            'rows': rows,
            'start': i, 
            'search-type': 'keyword',
            'request-id': '3998633988543',
            'storeid': 3132, 
            'pagename': 'search',
            'url': 'www.safeway.com', 
            'pageurl': 'www.safeway.com', 
            'dvid': 'web-autosuggest', 
            'facet': 'false',
            'visitorId': ""
        }
        headers = {'ocp-apim-subscription-key': 'e914eec9448c4d5eb672debf5011cf8f'}
        # get the information from safeway
        r = requests.get(url, params, headers = headers)
        # if no results were returned
        # escape the loop
        if not r.json()["response"]["docs"]:
            break
        # update list w/ all results
        all_res.append(r.json()["response"]["docs"])
        i += rows # move to next page
        print(i) # diagnostic statement
    return all_res

In [ ]:
# get the quantity and unit of measurement for each item
# which is after the dash
def extract_after_dash_regex(input_string):
    # Regular expression to find everything after " - "
    match = re.search(r'[–-]\s*(.+)', input_string)
    if match: # if the dash expression exists, return part after the dash
        return match.group(1)  # Return the part after " - "
    return ""  # Return an empty string if no match is found

# extract the quantity and unit of measurement from the string
def extract_quantity_and_unit(input_string):
    # Regular expression to match a quantity (number) followed by a unit of measurement (letters)
    pattern = r'(\d+(\.\d+)?)\s*([A-Za-z\.]+(?:\s[A-Za-z\.]+)?)'
    match = re.search(pattern, input_string) # find the qty and unit of measurement
    if match: # if the qty and unit of measurement exist, extract them
        quantity = float(match.group(1))  # Extract the quantity (number)
        unit = match.group(3)  # Extract the unit of measurement
        return quantity, unit
    return None, None  # Return None if no match is found

# get the search results from safeway
# and organize into a pandas dataframe
def get_saf_prod(ingredient:str):
    # get search results
    search_res = safeway_search(ingredient)
    # initialize list to store most important results
    search_list = []
    # run through search results and populate list
    for item in search_res:
        for subitem in item:
            name = subitem["name"] # product name
            price = subitem["basePrice"] # non-sale price
            # extract the qty and units of measurement
            qty_meas = extract_after_dash_regex(name)
            qty,unit = extract_quantity_and_unit(qty_meas)
            # organize results into list
            subitem_lst = [name,price,qty,unit]
            # update master list
            search_list.append(subitem_lst)
    # organize results into dataframe
    df = pd.DataFrame(data=search_list,columns=["Name","Price","Qty","Unit"])
    return df

### Butter

In [ ]:
# retrieve the butter products from safeway
butter = get_saf_prod("unsalted butter")

30
60


In [19]:
butter

,Name,Price,Qty,Unit
0,Lucerne Unsalted Sweet Cream Butter Quarters -...,5.99,16.0,Oz
1,Challenge Butter Unsalted - 16 Oz,8.99,16.0,Oz
2,Clover Butter Unsalted Sweet Cream 4 Count - 1...,7.99,16.0,Oz
3,Kerrygold Butter Pure Irish Two Sticks Unsalte...,6.49,8.0,Oz
4,Lucerne Butter Salted Sweet Cream 4 Quarters -...,5.99,16.0,Oz
5,Lucerne European Style Unsalted Butter - 8 Oz,3.99,8.0,Oz
6,Plugra European Style Unsalted Butter - 8 Oz,4.49,8.0,Oz
7,O Organics Organic Butter Sweet Cream Unsalted...,8.99,16.0,Oz
8,Kerrygold Butter Pure Irish Unsalted - 8 Oz,6.49,8.0,Oz
9,Kerrygold Butter Pure Irish Two Sticks - 8 Oz,6.49,8.0,Oz


In [ ]:
# filter for relevant results
butter_fil = butter[ # check enough quantity
    (((butter['Qty'] >= 8) & (butter['Unit'].str.contains('oz',case=False,na=False))) | # check for sufficient qty in oz
     ((butter['Qty'] >= 1) & (butter['Unit'].str.contains('lb',case=False,na=False)))) & # check for sufficient qty in lbs
    (butter['Name'].str.contains('Unsalted', case=False, na=False))
]
# retrieve the minimum price
min_but = butter_fil.sort_values(ascending=True,by="Price").head(1)

### Flour

In [ ]:
# retrieve search results for flour
flour = get_saf_prod("all-purpose flour")

30


In [22]:
flour

,Name,Price,Qty,Unit
0,Gold Medal Bleached Enriched Presifted All Pur...,6.99,5.0,Lb
1,Signature SELECT Flour All Purpose Pre-Sifted ...,4.99,5.0,Lb
2,King Arthur Baking Company Unbleached All Purp...,8.99,5.0,Lb
3,Signature SELECT Flour All Purpose Pre-Sifted ...,3.79,32.0,Oz
4,Signature SELECT Flour All Purpose Pre-Sifted ...,4.99,5.0,Lb
5,Gold Medal Flour All-Purpose Unbleached - 5 Lb,6.99,5.0,Lb
6,Gold Medal Bleached Enriched Presifted All Pur...,4.49,32.0,Oz
7,Bob's Red Mill All Purpose Unbleached White Fl...,5.99,5.0,Lb
8,Gold Medal Wondra Flour Quick-Mixing Enriched ...,4.99,13.5,Oz
9,Signature SELECT Flour All Purpose Pre-Sifted ...,8.79,10.0,Lb


In [ ]:
# filter for relevant results
flour_fil = flour[
    (flour['Name'].str.contains('Purpose Flour', case=False, na=False)) & # check that the product is actually flour
    ((flour["Unit"].str.contains("lb", case=False, na=False) & (flour["Qty"] >= 3*0.275578)) | # check for minimum ounces
     (flour["Unit"].str.contains("oz", case=False, na=False) & (flour["Qty"] >= 3*4.40925))) # check for minimum pounds
]
# get product with lowest price
min_flo = flour_fil.sort_values(by="Price",ascending=True).head(1)

### Sugar

In [ ]:
# get the sugar products
sugar = get_saf_prod("white sugar")

30
60


In [20]:
sugar

,Name,Price,Qty,Unit
0,Signature SELECT Granulated Sugar - 4 Lb (Pack...,5.49,4.00,Lb
1,C&H Granulated Pure Cane White Sugar Carton - ...,3.79,1.00,Lb
2,C&H Premium Pure Cane Granulated Sugar Bag - 4 Lb,5.99,4.00,Lb
3,C&H Premium Pure Cane Granulated Sugar - 2 Lb,4.29,2.00,Lb
4,O Organics Organic Sugar Granulated - 32 Oz,4.99,32.00,Oz
5,Signature SELECT Granulated Fine Sugar - 10 Lb,12.99,10.00,Lb
6,Signature SELECT Sugar Brown Light - 32 Oz,4.99,32.00,Oz
7,Signature SELECT Powdered Sugar Confectioners ...,3.79,16.00,Oz
8,C&H Premium Pure Cane Dark Brown Sugar Bag - 2 LB,5.49,2.00,LB
9,C&H Pure Cane Ultrafine Granulated Baker's Sug...,6.99,4.00,Lb


In [ ]:
# check for relevant products
sugar_fil = sugar[
    (sugar['Name'].str.contains('Granulated', case=False, na=False)) & # check for right type of sugar
    ((sugar["Unit"].str.contains("lb", case=False, na=False) & (sugar["Qty"] >= 0.440925)) | # check for minimum ounces
     (sugar["Unit"].str.contains("oz", case=False, na=False) & (sugar["Qty"] >= 7.054792))) # check for minimum pounds
]
# find the minimum sugar price
min_sug = sugar_fil.sort_values(by="Price",ascending=True).head(1)

### Brown Sugar

In [ ]:
# get the brown sugar products
brown = get_saf_prod("brown sugar")

30
60


In [27]:
brown

,Name,Price,Qty,Unit
0,Signature SELECT Sugar Brown Light - 32 Oz,4.99,32.00,Oz
1,Signature SELECT Sugar Brown Light - 16 Oz,3.79,16.00,Oz
2,C&H Premium Pure Cane Dark Brown Sugar Bag - 2 LB,5.49,2.00,LB
3,Signature SELECT Sugar Brown Dark - 32 Oz,4.99,32.00,Oz
4,Signature SELECT Sugar Brown Dark - 16 Oz,3.79,16.00,Oz
5,C&H Premium Pure Cane Light Brown Sugar Bag - ...,5.49,2.00,LB
6,O Organics Organic Sugar Brown Light - 24 Oz,5.99,24.00,Oz
7,C&H Premium Pure Cane Granulated Sugar Bag - 4 Lb,5.99,4.00,Lb
8,O Organics Sugar Dark Brown - 24 Oz,5.99,24.00,Oz
9,C&H Premium Cane Powdered Sugar - 2 LB,5.49,2.00,LB


In [ ]:
# check for relevant products
brown_fil = brown[
    (brown['Name'].str.contains('Brown', case=False, na=False)) & # check for right product
    ((brown["Unit"].str.contains("lb", case=False, na=False) & (brown["Qty"] >= 0.440925)) | # check for minimum ounces
     (brown["Unit"].str.contains("oz", case=False, na=False) & (brown["Qty"] >= 7.054792))) # check for minimum pounds
]
# retrieve the product w/ lowest price
min_bro = brown_fil.sort_values(by="Price",ascending=True).head(1)

### Eggs

In [ ]:
# get the egg products
eggs = get_saf_prod("large eggs")

30
60


In [ ]:
eggs

,Name,Price,Qty,Unit
0,Lucerne Farms Eggs Cage Free Large - 18 Count,9.59,18.0,Count
1,Lucerne Farms Eggs Large Cage Free - 12 Count,6.49,12.0,Count
2,Lucerne Cage Free Grade A Large Eggs - 60 Count,25.99,60.0,Count
3,Eggland's Best Cage Free Large White Eggs - 12...,5.49,12.0,Count
4,O Organics Organic Eggs Large Brown - 12 Count,6.49,12.0,Count
5,Eggland's Best Cage Free Large Brown Eggs - 12...,5.99,12.0,Count
6,Open Nature Eggs Brown Free Range Large - 18 C...,7.99,18.0,Count
7,Eggland's Best Cage Free Large White Eggs - 18...,7.99,18.0,Count
8,Open Nature Eggs Brown Free Range Large - 12 C...,5.99,12.0,Count
9,Lucerne Farms Eggs Cage Free Extra Large - 12 ...,6.69,12.0,Count


In [ ]:
# check for relevant products
eggs_fil = eggs[
    (eggs['Name'].str.contains('large', case=False, na=False)) & # check for right type of eggs
    # handle two different ways to write the number of eggs
    ((eggs["Unit"].str.contains("count", case=False, na=False) & (eggs["Qty"] >= 2)) |
     (eggs["Unit"].str.contains("ct", case=False, na=False) & (eggs["Qty"] >= 2)))
]
# get the lowest price
min_egg = eggs_fil.sort_values(by="Price",ascending=True).head(1)

### Vanilla Extract

In [ ]:
# get the vanilla extract search products
vanilla = get_saf_prod("vanilla extract")

30


In [32]:
vanilla

,Name,Price,Qty,Unit
0,McCormick Pure Vanilla Extract - 2 Fl. Oz.,9.99,2.0,Fl. Oz.
1,Signature SELECT Extract Pure Vanilla - 1 Fl. Oz.,3.99,1.0,Fl. Oz.
2,McCormick All Natural Pure Vanilla Extract - 1...,6.99,1.0,Fl. Oz.
3,Signature SELECT Extract Pure Vanilla - 2 Fl. Oz.,5.99,2.0,Fl. Oz.
4,Value Corner Imitation Vanilla Flavor - 8 Fl. Oz.,2.49,8.0,Fl. Oz.
5,McCormick All Natural Pure Vanilla Extract - 4...,17.99,4.0,Fl. Oz.
6,Morton & Bassett Extract Pure Vanilla - 4 Fl. Oz.,22.99,4.0,Fl. Oz.
7,Morton & Basset Premium Quality Pure Vanilla E...,13.49,2.0,Oz.
8,Signature SELECT Pure Vanilla Extract - 4 FZ,9.99,4.0,FZ
9,O Organics Organic Vanilla Extract Pure - 4 Fl...,17.49,4.0,Fl. Oz.


In [ ]:
# filter for relevant products
vanilla_fil = vanilla[
    vanilla['Name'].str.contains('Vanilla', case=False, na=False) & # need right product
    vanilla['Name'].str.contains('Extract', case=False, na=False) &
    (vanilla["Qty"] >= 2*.2) & vanilla["Unit"].str.contains("fl", case=False, na=False) # liquid form for the recipe
]
# get the lowest price
min_van = vanilla_fil.sort_values(by="Price",ascending=True).head(1)

### Baking Soda

In [ ]:
# get baking soda products
soda = get_saf_prod("baking soda")

30


In [36]:
soda

,Name,Price,Qty,Unit
0,ARM & HAMMER Baking Soda - 1 Lb,1.99,1.0,Lb
1,Clabber Girl Baking Powder - 8.1 Oz,4.79,8.1,Oz
2,Signature SELECT Baking Soda - 16 Oz,1.49,16.0,Oz
3,ARM & HAMMER Pure Baking Soda - 4 Lb,5.99,4.0,Lb
4,Signature SELECT Baking Powder Double Acting -...,2.49,8.1,Oz
5,ARM & HAMMER Fridge N Freezer Odor Absorber Ba...,2.79,14.0,Oz
6,Bobs Red Mill Baking Soda Gluten Free - 16 Oz,3.59,16.0,Oz
7,Rumford Baking Powder Reduced Sodium - 4 Oz,3.99,4.0,Oz
8,ARM & HAMMER Cat Litter Deodorizer - 20 Oz,3.99,20.0,Oz
9,ARM & HAMMER Waste Bag Refill Odor Control Wit...,10.99,90.0,Count


In [38]:
# filter for relevant products
soda_fil = soda[
    soda['Name'].str.contains('Baking Soda', case=False, na=False) & 
    (((soda["Qty"] >= .2) & soda["Unit"].str.contains("oz", case=False, na=False)) | # handle ounces case
    ((soda["Qty"] >= .0125) & soda["Unit"].str.contains("lb", case=False, na=False))) # handle lb case
]
# check for minimum price
min_sod = soda_fil.sort_values(ascending=True,by="Price").head(1)

### Semisweet Chocolate Chips

In [ ]:
# get chocolate chip products
chips = get_saf_prod("semisweet chocolate chips")

30


In [40]:
chips

,Name,Price,Qty,Unit
0,Nestle Toll House Semi Sweet Chocolate Mini Ch...,7.49,10.0,Oz
1,Guittard Baking Chips Semisweet Chocolate 46% ...,7.49,12.0,Oz
2,Signature SELECT Chocolate Chips Real Semi-Swe...,4.99,12.0,Oz
3,Ghirardelli Milk Chocolate Premium Baking Chip...,5.99,11.5,Oz
4,Ghirardelli 60% Cacao Bittersweet Chocolate Pr...,5.99,10.0,Oz
5,Nestle Toll House Semi Sweet Chocolate Chips -...,7.49,12.0,Oz
6,Ghirardelli Premium Baking Semi-Sweet Chocolat...,5.99,12.0,Oz.
7,Signature SELECT Chocolate Chips Milk Chocolat...,4.99,11.5,Oz
8,Guittard Baking Chips Milk Chocolate 31% Cacao...,7.49,11.5,Oz
9,Nestle Toll House Semi Sweet Chocolate Chips -...,11.79,24.0,Oz


In [ ]:
# filter for relevant products
chips_fil = chips[
    chips['Name'].str.contains('semi', case=False, na=False) & 
    ((chips["Qty"] >= 12) & chips["Unit"].str.contains("oz", case=False, na=False)) # handle oz case
]
# check for minimum price
min_cho = chips_fil.sort_values(ascending=True,by="Price").head(1)

### Chopped Walnuts

In [ ]:
# get search results for chopped walnuts
nuts = get_saf_prod("chopped walnuts")

30
60
90


In [14]:
nuts

,Name,Price,Qty,Unit
0,Diamond of California Chopped Walnuts – 14 Oz.,10.99,14.00,Oz.
1,Open Nature Walnuts Chopped Bag - 8 Oz,4.99,8.00,Oz
2,Diamond of California Walnuts Chopped - 8 Oz,7.99,8.00,Oz
3,Diamond of California Walnuts Chopped - 2.25 Oz,4.49,2.25,Oz
4,Open Nature Walnuts Chopped Bag - 2.3 Oz,3.49,2.30,Oz
5,Diamond of California Chopped Pecans – 8 Oz.,9.99,8.00,Oz.
6,Diamond Walnuts Shelled - 16 Oz,9.99,16.00,Oz
7,Signature Select/Farms Walnut Halves - 16 Oz,5.99,16.00,Oz
8,Diamond of California Walnuts Finely Diced - 1...,8.99,10.00,Oz
9,Diamond of California Nut Toppings 100% Walnut...,4.49,2.25,Oz


In [ ]:
# filter for relevant products
nuts_fil = nuts[
    nuts['Name'].str.contains('walnut', case=False, na=False) & 
    nuts['Name'].str.contains('chopped', case=False, na=False) &
    (((nuts["Qty"] > .275) & nuts["Unit"].str.contains("lb", case=False, na=False)) | # handle lb case
    ((nuts["Qty"] >= 4.4) & nuts["Unit"].str.contains("oz", case=False, na=False))) # handle ounces case
]
# check for minimum price
min_nut = nuts_fil.sort_values(ascending=True,by="Price").head(1)

### Total Price

In [ ]:
# organize all results to one dataframe
safeway = pd.concat([min_van,min_bro,min_but,min_cho,min_egg,min_flo,min_nut,min_sod,min_sug])

In [ ]:
# calculate total price
safeway["Price"].sum()

34.510000000000005